In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
from hyperopt import fmin, tpe, hp, Trials, space_eval
from sklearn.metrics import mean_squared_error
from util import evaluate, plot_importance, plot_history
from sklearn.model_selection import train_test_split#安装库运行环境先百度清华镜像，清华镜像下载速度较快

In [ ]:
data = pd.read_csv(r"G:\数据准备\建成环境\北京市\建成环境计算结果2\北京2.csv")

In [ ]:
cols = ['build_buguize', 'avg_height', 'shape_coefficient_sum', 'jiaochakou', 'mean_kuai', 'shape_index',
             'TotalDensity', 'duanbofushe', 'rizhaodanweishu', 'wendu', 'gdp', 'dengguang', 'ndvi', 'mean_SVF',
             'zuigaoqiwen', 'ydl', 'beijing_LD_midu', 'beijing_SY_midu', 'beijing_TCC_midu', 'poi_hhd', 'p_cd',
             'p_gyyd', 'p_gggl', 'p_gc', 'p_jt', 'p_ld', 'p_nd', 'p_sl', 'p_sy', 'p_sd', 'p_st', 'p_zzyd',
             'landuse_hhd', 'road_density', 'bus_coverage_area', 'subway_coverage_area', 'floor_area_ratio']#选择的自变量
X = data[cols]
y = data['value_x']

In [ ]:
# 定义名字映射
name_mapping = {
    'beijing_GF_midu': 'DPSF',
    'beijing_JT_midu': 'DTF',
    'beijing_JZ_midu': 'DRB',
    'beijing_LD_midu': 'DGF',
    'beijing_SY_midu': 'DCF',
    'beijing_TCC_midu': 'PLD',
    'poi_hhd': 'MUOI',
    'road_density': 'RD',
    'population_density': 'DP',
    'bus_coverage_area': 'BCM',
    'subway_coverage_area': 'SSCM',
    'building_density': 'BD',
    'floor_area_ratio': 'FAR',
    'p_cd': 'GF',
    'p_gyyd': 'PGF',
    'p_gggl': 'AF',
    'p_gc': 'SF',
    'p_jt': 'TSF',
    'p_ld': 'BLF',
    'p_nd': 'CF',
    'p_sl': 'FF',
    'p_sy': 'CSF',
    'p_sd': 'WF',
    'p_st': 'WBF',
    'p_zzyd': 'RF',
    'landuse_hhd': 'MULT',
    'ndvi': 'NDVI',
    'duanbofushe': 'ASWR',
    'fengsu': 'AMWP',
    'jiangshuiliang': 'AMPr',
    'pingjundiwen': 'AMGT',
    'qiya': 'AMPa',
    'rizhaodanweishu': 'AAS',
    'wendu': 'AMT',
    'shidu': 'AMRH',
    'zhengfaliang': 'AE',
    'build_buguize': 'BBI',
    'avg_height': 'MBH',
    'std_dev_height': 'SDBH',
    'shape_coefficient_sum': 'BSC',
    'jiaochakou': 'NRD',
    'shape_index': 'MSI',
    'TotalDensity': 'ED',
    'mean_SVF': 'SVF',
    'gdp': 'GDP',
    'zuigaoqiwen': 'AMMTmax',
    'zuidiqiwen': 'AMMTmin',
    'mean_kuai': 'PD',
    'dengguang': 'NLD',
    'ydl':'EC'
}

# 应用名字映射到data
data.rename(columns=name_mapping, inplace=True)

# 打印前几行检查新列名
print(data.head())

# 更新选择的自变量列名（如果需要）
cols_mapped = [name_mapping.get(col, col) for col in cols]  # 使用映射名称，如果没有映射则保留原名

# 提取更新后的自变量和因变量
X = data[cols_mapped]  # 这里使用更新后的列名
y = data['value_x']  # 因变量不需要重命名

# 打印X的列名以确认更新正确
print("更新后的X列名:", X.columns.tolist())

In [ ]:
X

In [ ]:
y

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)#总数据分出0.2测试集和0.8训练集

In [ ]:
X_train,X_valid,y_train,y_valid = train_test_split(X_train,y_train,test_size=0.2,random_state=42)#0.8的训练集分成0.2的训练集和0.8测试集

In [ ]:
print('训练集大小:',X_train.shape[0])
print('测试集大小:',X_test.shape[0])
print('验证集大小:',X_valid.shape[0])

In [ ]:
param_space = {
    "max_depth": hp.choice("max_depth", np.linspace(1, 10, 10, dtype=int)),#2是最小，10是最大，9是分9份。此参数为学习深度默认值6典型最终值3-10
    "gamma": hp.choice("gamma", np.linspace(0.01, 1, 10)),#默认值为0使算法保守根据损失函数变化进行调整
    "n_estimators": hp.choice("n_estimators", np.linspace(1, 500, 500, dtype=int)),#最后是分多少份，迭代可以增加平线长度使得测试集和训练集更加贴近，老师修改的4000
    "colsample_bytree": hp.choice("colsample_bytree", np.linspace(0.5, 1, 10)),#减慢学习速度方法通过子样本和这个减少样本数和特征，让数值更小典型最终值0.5-1
    "subsample": hp.quniform("subsample", 0.5, 0.7, 0.1),#使算法更加保守，防治过拟合，太小的值会欠拟合。典型最终值0.5-1
    "eta": hp.uniform("eta", 0.01, 0.2),#学习率可以放慢学习速度，限制集合中每棵树的贡献默认值0.3，可以尝试较小的值0.01，典型最终值0.01-0.2
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "random_state": 0
}

In [ ]:
def score(params):
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train,#可以修改为训练集、测试集、验证集，寻找最佳参数的某个值
              eval_set=[(X_train, y_train), (X_valid, y_valid)],
              early_stopping_rounds=10,
              verbose=False)
    y_pred = model.predict(X_valid)
    score = mean_squared_error(y_valid, y_pred)
    return {"loss": score, "status": "ok"}

In [ ]:
trials = Trials()
best_params_loc = fmin(score, param_space, algo=tpe.suggest, max_evals=50, trials=trials)
optimal_params = space_eval(param_space, best_params_loc)

In [ ]:
optimal_params#输出最佳参数

# Fit on entire train and evaluate on test set

In [ ]:
X_train = pd.concat([X_train, X_valid])
y_train = pd.concat([y_train, y_valid])

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [ ]:
model = xgb.XGBRegressor(**optimal_params)
model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test), (X_valid, y_valid)],
    early_stopping_rounds=10,
    verbose=True
)

In [ ]:
Z = model.predict(data[cols_mapped])

xx = np.zeros(shape=(1573,2))
xx[:,0]=data['value_x'].values
xx[:,1]=Z
np.savetxt('XGB.csv', xx, delimiter = ',') 

In [ ]:
y_train_pred = model.predict(X_train)
evaluate(y_train.to_numpy(), y_train_pred)

In [ ]:
y_valid_pred = model.predict(X_valid)
evaluate(y_valid.to_numpy(), y_valid_pred)

In [ ]:
y_pred = model.predict(X_test)
evaluate(y_test.to_numpy(), y_pred)

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
# define an additional metric 
def MAPE(true, pred):
    diff = np.abs(np.array(true) - np.array(pred))
    return np.mean(diff / abs(true))

In [ ]:
print("Training R2:", r2_score(y_train, y_train_pred), "traing RMSE:", np.sqrt(mean_squared_error(y_train, y_train_pred)), "training MAE:", mean_absolute_error(y_train, y_train_pred), "MAPE:", MAPE(y_train, y_train_pred)*100,"%")

In [ ]:
print("Testing R2:", r2_score(y_test, y_pred), "Testing RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)), "Testing MAE:", mean_absolute_error(y_test, y_pred), "MAPE:", MAPE(y_test, y_pred)*100,"%")

In [ ]:
print("Valid R2:", r2_score( y_valid, y_valid_pred), "valid RMSE:", np.sqrt(mean_squared_error(y_valid, y_valid_pred)), "Testing MAE:", mean_absolute_error(y_valid, y_valid_pred), "MAPE:", MAPE(y_valid, y_valid_pred)*100,"%")

In [ ]:
import os
import numpy as np
import pandas as pd
import xgboost as xgb
import seaborn as sns
from tabulate import tabulate
import matplotlib.pyplot as plt
from scipy.stats import probplot
from statsmodels.stats.stattools import jarque_bera
from sklearn import metrics


def combine_data():
    data = []
    for file in sorted(os.listdir("pp_gas_emission")):
        df = pd.read_csv(f"./pp_gas_emission/{file}")
        df['year'] = int(file[3:7])
        data.append(df)
    df = pd.concat(data)
    df.drop_duplicates(inplace=True)
    df.reset_index(inplace=True, drop=True)
    df["target"] = df["TEY"].shift(-1)
    df.drop(df.tail(1).index, inplace=True)
    df.to_parquet("pp_gas_emission.parquet")


def evaluate(y_test, y_pred):
    mae = metrics.mean_absolute_error(y_test, y_pred)
    mse = metrics.mean_squared_error(y_test, y_pred)
    rmse = metrics.mean_squared_error(y_test, y_pred, squared=False)
    r2 = metrics.r2_score(y_test, y_pred)
    explained_var_score = metrics.explained_variance_score(y_test, y_pred)
    mape = metrics.mean_absolute_percentage_error(y_test, y_pred)
    print(f"MAE = {mae}")
    print(f"MSE = {mse}")
    print(f"RMSE = {rmse}")
    print(f"MAPE = {mape}")
    print(f"R^2 = {r2}")
    print(f"Explained Variance Score = {explained_var_score}")

    residuals = y_pred - y_test
    print("\nResiduals summary stats")
    print(tabulate(
        pd.Series(residuals).describe().to_frame(),
        numalign="right", tablefmt="fancy_grid",
    ))

    with sns.axes_style("darkgrid"):
        g = sns.jointplot(x=y_pred, y=y_pred-y_test, s=8, alpha=0.75, height=12)
        g.set_axis_labels('predicted', 'predicted - actual')
        g.fig.set_figwidth(15)
        g.fig.set_figheight(6)
        g.ax_marg_x._visible = False

        plt.figure(figsize=(10, 8))
        g = sns.jointplot(x=y_pred, y=y_test, s=8, alpha=0.75, height=12)
        g.set_axis_labels('predicted', 'actual')

        fig, ax = plt.subplots(1, 1, figsize=(6, 10))
        probplot(residuals, sparams=(0, 1), plot=ax, fit=False)
        ax.set_title("Residuals probability plot")

        plt.figure(figsize=(20, 5))
        plt.plot(y_pred, alpha=0.5, label="prediction")
        plt.plot(y_test, alpha=0.5, label="actual")
        plt.legend()
        plt.title("Test set prediction over time")
        
        plt.figure(figsize=(20, 5))
        plt.plot(100*(residuals)/y_test, linewidth=0.5)
        plt.title("Test set percentage error over time")
    
    res_mean = residuals.mean()
    res_std = residuals.std()

    one_std_condition = ((residuals >= res_mean - 1*res_std) & (residuals <= res_mean + 1*res_std))
    two_std_condition = ((residuals >= res_mean - 2*res_std) & (residuals <= res_mean + 2*res_std))
    three_std_condition = ((residuals >= res_mean - 3*res_std) & (residuals <= res_mean + 3*res_std))
    perc_within_one_std = 100*len(residuals[np.where(one_std_condition)])/len(residuals)
    perc_within_two_std = 100*len(residuals[np.where(two_std_condition)])/len(residuals)
    perc_within_three_std = 100*len(residuals[np.where(three_std_condition)])/len(residuals)

    print("\nDeviation from mean of residuals")
    print(tabulate(pd.DataFrame({
        "μ ± σ": {"actual": perc_within_one_std, "expected": 68.2},
        "μ ± 2σ": {"actual": perc_within_two_std, "expected": 95.4},
        "μ ± 3σ": {"actual": perc_within_three_std, "expected": 99.7},  
    }).T, headers=["Interval", "Actual", "Expected"], tablefmt="fancy_grid"))
    
    print("\nJarque-Bera Test on Residuals")
    result = jarque_bera(residuals)
    result_labels = ("JB-Statistic", "Chi2-P-value", "Skew", "Kurtosis")
    print(tabulate(list(zip(result_labels, result)), tablefmt="fancy_grid"))


def plot_importance(model):
    # with sns.set_style("whitegrid"):
    with plt.style.context('ggplot'):
        fig, axes = plt.subplots(1, 3, figsize=(18, 18))
        fig.tight_layout(pad=8)
        axes[0] = xgb.plot_importance(model, ax=axes[0],
                                      importance_type="weight",
                                      xlabel="weight")
        axes[1] = xgb.plot_importance(model, ax=axes[1],
                                      importance_type="gain",
                                      xlabel="gain")
        axes[2] = xgb.plot_importance(model, ax=axes[2],
                                      importance_type="cover",
                                      xlabel="cover")
        for ax, title in zip(axes, ["weight", "gain", "cover"]):
            ax.set_title(f"Features ranked by {title}")
            ax.set_ylabel(None)
            ax.tick_params(axis="x", labelsize=10)
            ax.tick_params(axis="y", labelsize=10)
            for txt_obj in ax.texts:
                curr_text = txt_obj.get_text()
                new_text = round(float(curr_text), 1)
                txt_obj.set_text(new_text)
    return


def plot_history(model):
    # with sns.set_style("whitegrid"):
    with plt.style.context('ggplot'):
        history = model.evals_result()
        fig, ax = plt.subplots(1, 1, figsize=(14, 5))
        ax.plot(range(len(history['validation_0']['rmse'])),
                history['validation_0']['rmse'],
                label="Training RMSE", linewidth=2)
        ax.plot(range(len(history['validation_1']['rmse'])),
                history['validation_1']['rmse'],
                label="Testing RMSE", linewidth=2)
        ax.set_title('RMSE')
        ax.grid(True)
        ax.legend()
    return


def rolling_means(df, window, min_periods, cols=None):
    cols = cols or df.columns.to_list()
    df_roll = (df[cols]
               .rolling(window, min_periods=min_periods)
               .mean()
               .fillna(df))
    df_roll.columns = df_roll.columns.map(lambda c: f"{c}_ROLL{window}_MEAN")
    return df_roll


def rolling_standard_deviation(df, window, min_periods, cols=None):
    cols = cols or df.columns.to_list()
    df_roll = (df[cols]
               .rolling(window, min_periods=min_periods)
               .std()
               .fillna(1))
    df_roll.columns = df_roll.columns.map(lambda c: f"{c}_ROLL{window}_STD")
    return df_roll


def lagged_features(df, lags, cols=None):
    if isinstance(lags, int):
        lags = range(1, lags+1)
    cols = cols or df.columns.to_list()
    data = []
    for lag in lags:
        lagged_data = df[cols].shift(lag).fillna(df)
        lagged_data.columns = lagged_data.columns.map(lambda c: f"{c}_LAG{lag}")
        data.append(lagged_data)
    return pd.concat(data, axis="columns")

In [ ]:
plot_history(model)

In [ ]:
plot_importance(model)

In [ ]:
import shap

In [ ]:
cols = ['BBI', 'MBH', 'BSC', 'NRD', 'PD', 'MSI', 'ED', 'ASWR', 'AAS', 'AMT', 'GDP', 'NLD', 'NDVI', 'SVF', 'AMMTmax', 'EC', 'DGF', 'DCF', 'PLD', 'MUOI', 'GF', 'PGF', 'AF', 'SF', 'TSF', 'BLF', 'CF', 'FF', 'CSF', 'WF', 'WBF', 'RF', 'MULT', 'RD', 'BCM', 'SSCM', 'FAR']

In [ ]:
explainer = shap.TreeExplainer(model)#这里创建了一个TreeExplainer对象，它用于解释基于树的模型（如决策树、随机森林或梯度提升机）。model是指已经训练好的模型实例。
shap_values = explainer.shap_values(data[cols])#调用shap_values()方法计算给定数据集data[cols]（其中cols是特征列名列表）的SHAP值。SHAP值衡量了每个特征对单个预测的贡献程度。
                                          #SHAP值是一个数组，其形状通常是(n_outputs, n_samples, n_features)。如果您的模型只有一个输出，则形状为(n_samples, n_features)
print(shap_values.shape)#打印出SHAP值数组的形状，以帮助了解其结构和维度
np.savetxt('shap values.csv', shap_values, delimiter = ',') #将计算得到的SHAP值保存到一个CSV文件中。请注意，如果shap_values包含多个输出，则可能需要分别保存每个输出的SHAP值，或者选择要保存的具体部分。
y_base = explainer.expected_value #获取基础预测值（expected value），这是在没有任何特征信息的情况下模型的平均预测值。对于回归问题，这通常等于训练数据目标变量的均值；对于分类问题，它是每个类别的先验概率
print(y_base)  #打印基础预测值，这对于理解模型的整体趋势很有帮助
data['pred'] = model.predict(data[cols]) #使用训练好的模型对data[cols]中的数据进行预测，并将预测结果作为新列'pred'添加到data DataFrame中。
print(data['pred'].mean()) #计算并打印预测结果的平均值。这个平均值可以与基础预测值（y_base）进行比较，以了解模型的总体偏移量或偏差

In [ ]:
import shap
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

# 假设shap_values和data[cols]已经定义
# explainer = shap.TreeExplainer(model)
# shap_values = explainer.shap_values(data[cols])

plt.rcParams['font.family'] = 'Times New Roman'  # 设置全局字体为Times New Roman

# 创建图形和子图布局
fig = plt.figure(figsize=(14, 8))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 9], wspace=0.05)

# 在右侧为主图生成summary plot，注意使用color_bar=False来避免默认的颜色条
plt.subplot(gs[1])
shap.summary_plot(shap_values, data[cols], feature_names=data[cols].columns.tolist(), plot_type="dot", cmap="viridis", color_bar=False, show=False)

# 获取当前Axes实例，并进行调整
ax_main = plt.gca()
ax_main.yaxis.tick_right()  # 将y轴刻度移动到右侧
ax_main.yaxis.set_label_position("right")  # 设置y轴标签位置为右侧

# 移除y轴刻度线（非文本标签前的线）
ax_main.tick_params(axis='y', which='both', length=0)  # 设置y轴刻度线长度为0

# 左侧为颜色条
cbar_ax = fig.add_subplot(gs[0])
norm = plt.Normalize(vmin=shap_values.min(), vmax=shap_values.max())
sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)
sm.set_array([])

# 添加颜色条到左侧
fig.colorbar(sm, cax=cbar_ax, orientation='vertical')

# 调整颜色条的宽度
cbar_ax.set_aspect(20)  # 调整颜色条的宽高比

# 手动添加颜色条的标签，并设置字体
cbar_ax.text(0.5, 1.02, 'High', transform=cbar_ax.transAxes, ha='center', fontname='Times New Roman')
cbar_ax.text(-0.8, 0.5, 'Feature value', transform=cbar_ax.transAxes, ha='center', rotation=90, fontname='Times New Roman')
cbar_ax.text(0.5, -0.02, 'Low', transform=cbar_ax.transAxes, ha='center', fontname='Times New Roman')

# 显示图形
plt.show()

In [ ]:
# 设置matplotlib参数以更改字体和DPI
plt.rcParams['font.family'] = 'Times New Roman'  # 设置全局字体为Times New Roman
dpi_setting = 1000  # 设置DPI为300

# 使用data[cols]的列名作为feature_names，并创建带有指定DPI和字体的SHAP summary plot
fig, ax = plt.subplots(figsize=(10, 8), dpi=dpi_setting)  # 调整图形大小和DPI
shap.summary_plot(shap_values, data[cols], feature_names=data[cols].columns, plot_type="dot", cmap="viridis", show=False)
#ax.set_title('SHAP Summary Plot', fontsize=14, fontname='Times New Roman')  # 自定义标题字体

In [ ]:
#shap.summary_plot(shap_values, data[cols])

In [ ]:
from matplotlib import gridspec

In [ ]:
# 设置matplotlib参数以更改全局字体为Times New Roman
plt.rcParams['font.family'] = 'Times New Roman'  # 全局设置字体

dpi_setting = 1000  # 设置DPI为100
fig = plt.figure(figsize=(5000, 5), dpi=dpi_setting)  # 调整整个图形的尺寸，例如(20, 8)

gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1])  # 如果需要不同宽度的子图，可以调整这些比例

# Bar类型的SHAP summary plot放在左边
ax_left = fig.add_subplot(gs[0])
cmap = plt.get_cmap('viridis')
light_yellow = cmap(0.92)  # 选择色图中接近末端的颜色获取淡黄色
shap.summary_plot(shap_values, data[cols], feature_names=data[cols].columns, plot_type="bar", color=light_yellow, show=False)
ax_left.set_title('Bar Summary Plot', fontsize=14)

# Dot类型的SHAP summary plot放在右边
ax_right = fig.add_subplot(gs[1])
shap.summary_plot(shap_values, data[cols], feature_names=data[cols].columns, plot_type="dot", cmap="viridis", show=False)
ax_right.set_title('Dot Summary Plot', fontsize=14)

# 清除右侧子图的左侧标签
ax_right.set_yticklabels([])

# 获取x轴标签的位置和文本
xtick_locs = ax_left.get_xticks()
xtick_labels = ax_left.get_xticklabels()

        
# 调整子图之间的间距以及它们与图形边缘的距离
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1, wspace=0.8)  # 增加wspace值来加大两个图之间的间隔

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import spearmanr

# 假设 shap_values 和 data[cols] 已经定义
# 目标变量的实际值存储在名为 value 的变量中

# 计算每个特征的平均SHAP值绝对值作为重要性
feature_importance_df = pd.DataFrame({
    'Feature': X.columns.tolist(),
    'Importance': np.abs(shap_values).mean(axis=0)
})

# 计算每个特征的SHAP值与目标变量之间的斯皮尔曼等级相关系数
sr_correlations = []
for feature_idx in range(len(cols)):
    correlation, _ = spearmanr(shap_values[:, feature_idx], y)  # 使用 y 替换 value
    sr_correlations.append(correlation)

# 将计算得到的相关系数添加到DataFrame中
feature_importance_df['SR_Correlation'] = sr_correlations

# 按照 'Importance' 列降序排序
feature_importance_df_sorted = feature_importance_df.sort_values(by='Importance', ascending=False)

# 创建一个颜色映射，根据相关系数来设置颜色
norm = plt.Normalize(vmin=-1, vmax=1)  # 将颜色映射的范围设置为 -1 到 1
cmap = plt.get_cmap("coolwarm")  # 使用"coolwarm"色带

# 绘制柱状图
fig, ax = plt.subplots(figsize=(10, 6), dpi=1200)
bars = ax.barh(feature_importance_df_sorted['Feature'], feature_importance_df_sorted['Importance'], 
               color=cmap(norm(feature_importance_df_sorted['SR_Correlation'])))

# 设置轴标签和标题
ax.set_xlabel('Mean(|SHAP value|) (average impact on model output magnitude)', fontweight='bold', fontsize=14)
ax.set_ylabel('Features', fontweight='bold', fontsize=14)
ax.set_title('Feature Importance Visualization with SR Correlation', fontweight='bold', fontsize=16)

ax.tick_params(axis='x', labelsize=12, labelcolor='black', length=6)
ax.tick_params(axis='y', labelsize=12, labelcolor='black', length=6)

# 关闭右边和顶部的边框
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# 添加颜色条 (colorbar)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # 为空数组以创建colorbar
cbar = fig.colorbar(sm, ax=ax, label='SR Correlation', pad=0.13)

# 为每个柱状图添加相关系数的具体值
for i, bar in enumerate(bars):
    correlation = feature_importance_df_sorted.iloc[i]['SR_Correlation']
    ax.text(bar.get_width() + 0.05, bar.get_y() + bar.get_height()/2, f"r = {correlation:.3f}",
            va='center', ha='left', color='black', fontsize=10, fontweight='bold')

# 显示图表
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import shap

# 假设shap_values, data和cols已经正确定义

# 设置matplotlib参数以更改全局字体为Times New Roman
plt.rcParams['font.family'] = 'Times New Roman'  # 全局设置字体

# 从色图中选择一个位置来获得特定的颜色，例如选择0.8的位置来获取淡黄色
cmap = plt.get_cmap('viridis')
light_yellow = cmap(0.92)  # 0到1之间的浮点数，数值越大颜色越接近黄色

# 计算每个特征的重要性（这里我们使用平均绝对SHAP值）
feature_importance = np.abs(shap_values).mean(0)  # 使用np.abs计算绝对值，并求平均

# 获取排序索引，以确保bar plot和text标签按照重要性从小到大排列
sorted_indices = np.argsort(feature_importance)  # 按升序排列
sorted_feature_importance = feature_importance[sorted_indices]
sorted_feature_names = np.array(cols)[sorted_indices]

# 创建一个新的figure和axes对象
fig, ax = plt.subplots(figsize=(10, 8), dpi=1000)

# 绘制水平条形图
y_pos = np.arange(len(sorted_feature_names))
ax.barh(y_pos, sorted_feature_importance, color=light_yellow)
ax.set_yticks(y_pos)
ax.set_yticklabels([])  # 隐藏y轴标签，但保留y轴

# 移除图表的顶部和右侧边框
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(True)  # 显示左侧边框，即y轴
ax.spines['bottom'].set_visible(True)  # 保留底部边框

# 在每个条形图上添加具体数值
for i, v in enumerate(sorted_feature_importance):
    ax.text(v + 0.01, i, f'{v:.3f}', color='black', va='center')  # 文本稍微偏移以避免被遮挡

# 设置轴标签和标题等
ax.set_xlabel('mean(|SHAP value|) (average impact on model output magnitude)', fontweight='bold', fontsize=14)
ax.set_title('Feature Importance Visualization with SHAP Values', fontweight='bold', fontsize=16)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import shap


# 设置全局字体和字号，确保不加粗
plt.rcParams.update({
    'font.family': 'Times New Roman',
    'font.size': 12,
    'axes.labelweight': 'normal',  # 确保轴标签不加粗
    'axes.titleweight': 'normal',  # 确保标题不加粗
})

# 假设shap_values, data和cols已经正确定义

# 创建一个新的figure和使用GridSpec进行布局
fig = plt.figure(figsize=(24, 8))  # 调整整个figure的大小
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 2.5], wspace=0.5)  # 定义两个宽度不同的子图

# 第一个子图（SHAP summary plot）
ax1 = plt.subplot(gs[0])
shap.summary_plot(shap_values, data[cols], feature_names=data[cols].columns.tolist(), plot_type="dot", cmap="viridis", color_bar=False, show=False)
ax_main = plt.gca()
ax_main.yaxis.tick_right()
ax_main.yaxis.set_label_position("right")
ax_main.tick_params(axis='y', which='both', length=0)

cbar_ax = fig.add_axes([0.01, 0.15, 0.01, 0.7])  # 在左侧添加颜色条
norm = plt.Normalize(vmin=shap_values.min(), vmax=shap_values.max())
sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)
sm.set_array([])
fig.colorbar(sm, cax=cbar_ax, orientation='vertical')
cbar_ax.text(0.5, 1.02, 'High', transform=cbar_ax.transAxes, ha='center')
cbar_ax.text(-1.5, 0.5, 'Feature value', transform=cbar_ax.transAxes, ha='center', rotation=90)
cbar_ax.text(0.5, -0.02, 'Low', transform=cbar_ax.transAxes, ha='center')

# 第二个子图（特征重要性条形图）
ax2 = plt.subplot(gs[1])
feature_importance = np.abs(shap_values).mean(0)
sorted_indices = np.argsort(feature_importance)
sorted_feature_importance = feature_importance[sorted_indices]
sorted_feature_names = np.array(cols)[sorted_indices]

y_pos = np.arange(len(sorted_feature_names))

# 控制条形的宽度（height参数）
bar_height = 0.65  # 条形高度，值越小，条形越窄，条形间的间距越大

ax2.barh(y_pos, sorted_feature_importance, height=bar_height, color=plt.get_cmap('viridis')(0.92), align='center')
ax2.set_yticks(y_pos)
ax2.set_yticklabels([])

# 调整条形图之间的间距
ax2.set_ylim(-bar_height, len(sorted_feature_names) - 1 + bar_height)  # 根据条形高度调整Y轴范围

ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.spines['left'].set_visible(True)
ax2.spines['bottom'].set_visible(True)

for i, v in enumerate(sorted_feature_importance):
    ax2.text(v + 0.01, i, f'{v:.2f}', color='black', va='center')

#ax2.set_xlabel('mean(|SHAP value|) (average impact on model output magnitude)', labelpad=6)  # 减少pad值使标签靠近轴线，负值向左移动
# 设置X轴标签，并调整其位置
ax2.set_xlabel('mean(|SHAP value|) (average impact on model output magnitude)')
# 调整X轴标签的位置，第一个参数是X轴上的位置(0到1)，第二个参数是Y轴上的位置
# 例如，将X轴标签向左移动到X轴的40%位置处
ax2.xaxis.set_label_coords(0.45, -0.04)

ax2.set_title('')
plt.gcf().set_dpi(1000)
plt.show()

In [ ]:
# 假设shap_values, data和cols已经正确定义
# 定义要显示的特征数量
num_features = 22

# 创建一个新的figure和使用GridSpec进行布局
fig = plt.figure(figsize=(24, 8))  # 调整整个figure的大小
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 2.5], wspace=0.5)  # 定义两个宽度不同的子图

# 第一个子图（SHAP summary plot）
ax1 = plt.subplot(gs[0])
# 设置max_display为22以确保显示22个特征
shap.summary_plot(shap_values[:, :num_features], data[selected_cols], feature_names=data[selected_cols].columns.tolist(), plot_type="dot", cmap="viridis", color_bar=False, show=False, max_display=num_features)
ax_main = plt.gca()
ax_main.yaxis.tick_right()
ax_main.yaxis.set_label_position("right")
ax_main.tick_params(axis='y', which='both', length=0)

cbar_ax = fig.add_axes([0.01, 0.15, 0.01, 0.7])  # 在左侧添加颜色条
norm = plt.Normalize(vmin=shap_values.min(), vmax=shap_values.max())
sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)
sm.set_array([])
fig.colorbar(sm, cax=cbar_ax, orientation='vertical')
cbar_ax.text(0.5, 1.02, 'High', transform=cbar_ax.transAxes, ha='center')
cbar_ax.text(-1.5, 0.5, 'Feature value', transform=cbar_ax.transAxes, ha='center', rotation=90)
cbar_ax.text(0.5, -0.02, 'Low', transform=cbar_ax.transAxes, ha='center')

# 第二个子图（特征重要性条形图）保持不变
ax2 = plt.subplot(gs[1])
feature_importance = np.abs(shap_values).mean(0)
sorted_indices = np.argsort(feature_importance)[-num_features:]  # 只选择最重要的22个特征
sorted_feature_importance = feature_importance[sorted_indices]
sorted_feature_names = np.array(cols)[sorted_indices]

y_pos = np.arange(len(sorted_feature_names))

# 控制条形的宽度（height参数）
bar_height = 0.85  # 条形高度，值越小，条形越窄，条形间的间距越大

ax2.barh(y_pos, sorted_feature_importance, height=bar_height, color=plt.get_cmap('viridis')(0.92), align='center')
ax2.set_yticks(y_pos)
ax2.set_yticklabels([])

# 调整条形图之间的间距
ax2.set_ylim(-bar_height, len(sorted_feature_names) - 1 + bar_height)  # 根据条形高度调整Y轴范围

ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.spines['left'].set_visible(True)
ax2.spines['bottom'].set_visible(True)

for i, v in enumerate(sorted_feature_importance):
    ax2.text(v + 0.01, i, f'{v:.2f}', color='black', va='center')

ax2.set_xlabel('mean(|SHAP value|) (average impact on model output magnitude)')
ax2.xaxis.set_label_coords(0.45, -0.04)

plt.gcf().set_dpi(1000)
plt.show()

In [ ]:
# 定义要显示的特征数量
num_features = 39

# 定义selected_cols（假设它是data中前22个特征）
selected_cols = cols[:num_features]

# 创建一个新的figure和使用GridSpec进行布局
fig = plt.figure(figsize=(24, 8))  # 调整整个figure的大小
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 2.5], wspace=0.5)  # 定义两个宽度不同的子图

# 第一个子图（SHAP summary plot）
ax1 = plt.subplot(gs[0])
# 设置max_display为22以确保显示22个特征
shap.summary_plot(shap_values[:, :num_features], data[selected_cols], feature_names=data[selected_cols].columns.tolist(), plot_type="dot", cmap="viridis", color_bar=False, show=False, max_display=num_features)
ax_main = plt.gca()
ax_main.yaxis.tick_right()
ax_main.yaxis.set_label_position("right")
ax_main.tick_params(axis='y', which='both', length=0)

cbar_ax = fig.add_axes([0.01, 0.15, 0.01, 0.7])  # 在左侧添加颜色条
norm = plt.Normalize(vmin=shap_values.min(), vmax=shap_values.max())
sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)
sm.set_array([])
fig.colorbar(sm, cax=cbar_ax, orientation='vertical')
cbar_ax.text(0.5, 1.02, 'High', transform=cbar_ax.transAxes, ha='center')
cbar_ax.text(-1.5, 0.5, 'Feature value', transform=cbar_ax.transAxes, ha='center', rotation=90)
cbar_ax.text(0.5, -0.02, 'Low', transform=cbar_ax.transAxes, ha='center')

# 第二个子图（特征重要性条形图）保持不变
ax2 = plt.subplot(gs[1])
feature_importance = np.abs(shap_values).mean(0)
sorted_indices = np.argsort(feature_importance)[-num_features:]  # 只选择最重要的22个特征
sorted_feature_importance = feature_importance[sorted_indices]
sorted_feature_names = np.array(cols)[sorted_indices]

y_pos = np.arange(len(sorted_feature_names))

# 控制条形的宽度（height参数）
bar_height = 0.85  # 条形高度，值越小，条形越窄，条形间的间距越大

ax2.barh(y_pos, sorted_feature_importance, height=bar_height, color=plt.get_cmap('viridis')(0.92), align='center')
ax2.set_yticks(y_pos)
ax2.set_yticklabels([])

# 调整条形图之间的间距
ax2.set_ylim(-bar_height, len(sorted_feature_names) - 1 + bar_height)  # 根据条形高度调整Y轴范围

ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.spines['left'].set_visible(True)
ax2.spines['bottom'].set_visible(True)

for i, v in enumerate(sorted_feature_importance):
    ax2.text(v + 0.01, i, f'{v:.2f}', color='black', va='center')

ax2.set_xlabel('mean(|SHAP value|) (average impact on model output magnitude)')
ax2.xaxis.set_label_coords(0.45, -0.04)

plt.gcf().set_dpi(1000)
plt.show()

In [ ]:
# 设置matplotlib参数以更改全局字体为Times New Roman
plt.rcParams['font.family'] = 'Times New Roman'  # 全局设置字体

# 我们可以从色图中选择一个位置来获得特定的颜色，例如选择0.8的位置来获取淡黄色
cmap = plt.get_cmap('viridis')
light_yellow = cmap(0.92)  # 0到1之间的浮点数，数值越大颜色越接近黄色

# 设置DPI和图形大小
dpi_setting = 1000  # 设置DPI为300
fig, ax = plt.subplots(figsize=(10, 8), dpi=dpi_setting)  # 调整图形大小和DPI
# 使用data[cols]的列名作为feature_names，并创建带有指定淡黄色的SHAP summary plot (bar)
shap.summary_plot(shap_values, data[cols], plot_type="bar", color=light_yellow)

In [ ]:
import matplotlib.pyplot as plt

# 设置matplotlib参数以更改全局字体为Times New Roman
plt.rcParams['font.family'] = 'Times New Roman'  # 全局设置字体

# 计算每个特征的重要性，并仅选择前22个最重要的特征
feature_importance = np.abs(shap_values).mean(0)
sorted_indices = np.argsort(feature_importance)[::1][:22]  # 按重要性降序排序并选择前22个
sorted_feature_importance = feature_importance[sorted_indices]
sorted_feature_names = np.array(cols)[sorted_indices]

# 设置DPI和图形大小
dpi_setting = 1000  # 设置DPI为1000
fig, ax = plt.subplots(figsize=(10, 8), dpi=dpi_setting)  # 调整图形大小和DPI

# 我们可以从色图中选择一个位置来获得特定的颜色，例如选择0.8的位置来获取淡黄色
cmap = plt.get_cmap('viridis')
light_yellow = cmap(0.92)  # 0到1之间的浮点数，数值越大颜色越接近黄色

# 手动创建条形图
bars = ax.barh(sorted_feature_names, sorted_feature_importance, color=light_yellow)

# 在每个条形的末端添加文本标注
for bar in bars:
    width = bar.get_width()
    ax.text(width + 0.01, bar.get_y() + bar.get_height()/2, f'{width:.2f}',
            ha='center', va='center', color='black')

plt.xlabel('Feature Importance')  # 可根据需要调整标签
plt.title('Top 22 Feature Importances')  # 标题
plt.tight_layout()  # 自动调整子图参数，使之填充整个图像区域
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 假设shap_values和cols已经定义
# 示例数据
# 注意：你需要确保在运行此段代码前，已正确导入shap_values和定义了cols列表。

# 设置matplotlib参数以更改全局字体为Times New Roman
plt.rcParams['font.family'] = 'Times New Roman'  # 全局设置字体

# 计算每个特征的重要性，并仅选择前22个最重要的特征
feature_importance = np.abs(shap_values).mean(0)
sorted_indices = np.argsort(feature_importance)[::-1][:22]  # 使用[::-1]实现降序排序
sorted_feature_importance = feature_importance[sorted_indices]
sorted_feature_names = np.array(cols)[sorted_indices]

# 设置DPI和图形大小
dpi_setting = 1000  # 设置DPI为1000
fig, ax = plt.subplots(figsize=(10, 8), dpi=dpi_setting)  # 调整图形大小和DPI

# 我们可以从色图中选择一个位置来获得特定的颜色，例如选择0.92的位置来获取淡黄色
cmap = plt.get_cmap('viridis')
light_yellow = cmap(0.92)  # 0到1之间的浮点数，数值越大颜色越接近黄色

# 手动创建条形图
bars = ax.barh(sorted_feature_names, sorted_feature_importance, color=light_yellow)

# 在每个条形的末端添加文本标注
for bar in bars:
    width = bar.get_width()
    ax.text(width + 0.01, bar.get_y() + bar.get_height()/2, f'{width:.2f}',
            ha='center', va='center', color='black')

plt.xlabel('Feature Importance')  # 可根据需要调整标签
plt.title('Top 22 Feature Importances')  # 标题
plt.tight_layout()  # 自动调整子图参数，使之填充整个图像区域
plt.show()

# 打印每个变量的名称和其对应的SHAP值
print("\nFeature names and their SHAP values (coefficient):")
for name, importance in zip(sorted_feature_names, sorted_feature_importance):
    print(f"Feature: {name}, SHAP Value (Coefficient): {importance:.2f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 示例数据
category_dict = {
    'BE': ['ASWR', 'AAS', 'FF', 'AMT', 'GF'],
    'SC': ['EC', 'GDP', 'CF', 'AMMTmax', 'BLF'],
    'LU': ['RD', 'NLD', 'FAR', 'WBF', 'NDVI'],
    'BC': ['AF', 'PGF', 'RF', 'TSF', 'BCM'],
    'SS': ['MULT', 'SF']
}

# 假设shap_values和cols已经定义
# 注意：你需要确保在运行此段代码前，已正确导入shap_values和定义了cols列表。
shap_values = np.random.rand(100, len(cols))  # 示例SHAP值

# 设置matplotlib参数以更改全局字体为Times New Roman
plt.rcParams['font.family'] = 'Times New Roman'

# 计算每个特征的重要性，并根据类别进行分组
category_importance = {cat: np.abs(shap_values[:, [cols.index(f) for f in features]]).mean() for cat, features in category_dict.items()}
sorted_categories = sorted(category_importance.keys(), key=lambda x: -category_importance[x])
sorted_importance = [category_importance[cat] for cat in sorted_categories]

# 根据类别给特征分组并生成对应的颜色
category_colors = {'BE': plt.cm.Reds(np.linspace(0.4, 1, len(category_dict['BE']))),
                   'SC': plt.cm.Blues(np.linspace(0.4, 1, len(category_dict['SC']))),
                   'LU': plt.cm.Greens(np.linspace(0.4, 1, len(category_dict['LU']))),
                   'BC': plt.cm.Purples(np.linspace(0.4, 1, len(category_dict['BC']))),
                   'SS': plt.cm.Greys(np.linspace(0.4, 1, len(category_dict['SS'])))}

# 创建甜甜圈图
fig, ax = plt.subplots(figsize=(10, 7), dpi=1000)

# 绘制甜甜圈图
wedges, texts, autotexts = ax.pie(sorted_importance, labels=sorted_categories,
                                  autopct='%1.1f%%', startangle=90, pctdistance=0.85,
                                  wedgeprops=dict(width=0.3, linewidth=2, edgecolor='black'), colors=[category_colors[cat][0] for cat in sorted_categories])

# 画圆心为空的圆，创造甜甜圈效果
centre_circle = plt.Circle((0,0),0.70,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

# Equal aspect ratio ensures that pie is drawn as a circle.
ax.axis('equal')  
plt.title('Feature Importances by Category')  # 标题

# 添加图例
legend_elements = []
for cat, features in category_dict.items():
    legend_elements.append(
        plt.Line2D([0], [0], marker='o', color='w', label=f'{cat} ({", ".join(features)})',
                   markerfacecolor=category_colors[cat][0], markersize=10)
    )
    for i, feature in enumerate(features):
        legend_elements.append(
            plt.Line2D([0], [0], marker='s', color='w', label=f'{feature}',
                       markerfacecolor=category_colors[cat][i], markersize=10)
        )

plt.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.15, 1))

plt.tight_layout()
plt.show()

# 打印每个变量的名称和其对应的SHAP值
print("\nFeature names and their SHAP values (coefficient):")
for name, importance in zip(cols, np.abs(shap_values).mean(axis=0)):
    print(f"Feature: {name}, SHAP Value (Coefficient): {importance:.2f}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 直接提供特征及其对应的SHAP值
feature_shap_values = [
    ('ASWR', 0.60), ('AAS', 0.51), ('FF', 0.24), ('AMT', 0.15),
    ('EC', 0.15), ('GF', 0.14), ('GDP', 0.13), ('CF', 0.12),
    ('AMMTmax', 0.10), ('NLD', 0.08), ('BLF', 0.08), ('RD', 0.08),
    ('FAR', 0.05), ('BCM', 0.05), ('NDVI', 0.04), ('AF', 0.04),
    ('TSF', 0.04), ('WBF', 0.03), ('MULT', 0.03), ('RF', 0.02),
    ('PGF', 0.02), ('SF', 0.00)
]

# 分类字典
category_dict = {
    'BE': ['ASWR', 'AAS', 'FF', 'AMT', 'GF'],
    'SC': ['EC', 'GDP', 'CF', 'AMMTmax', 'BLF'],
    'LU': ['RD', 'NLD', 'FAR', 'WBF', 'NDVI'],
    'BC': ['AF', 'PGF', 'RF', 'TSF', 'BCM'],
    'SS': ['MULT', 'SF']
}

# 根据类别给特征分组并生成对应的颜色
category_colors = {
    'BE': plt.cm.Reds(np.linspace(0.4, 1, len(category_dict['BE']))),
    'SC': plt.cm.Blues(np.linspace(0.4, 1, len(category_dict['SC']))),
    'LU': plt.cm.Greens(np.linspace(0.4, 1, len(category_dict['LU']))),
    'BC': plt.cm.Purples(np.linspace(0.4, 1, len(category_dict['BC']))),
    'SS': plt.cm.Greys(np.linspace(0.4, 1, len(category_dict['SS'])))
}

# 创建所有类别的大小和颜色
sizes = []
colors = []

for cat, features in category_dict.items():
    for feature in features:
        shap_value = next(val for name, val in feature_shap_values if name == feature)
        sizes.append(shap_value)
        colors.append(category_colors[cat][features.index(feature)])

# 创建甜甜圈图
fig, ax = plt.subplots(figsize=(10, 7), dpi=1000)

# 绘制细化的甜甜圈图
wedges, texts, autotexts = ax.pie(sizes, labels=None, autopct='%1.1f%%', startangle=90,
                                  wedgeprops=dict(width=0.3, edgecolor='black'), colors=colors)

# 外部标签仅显示主要类别
legend_labels = []
start_index = 0
for cat, features in category_dict.items():
    legend_labels.append(f'{cat} ({", ".join(features)})')
    end_index = start_index + len(features)
    # 设置主类别标签位置
    angle = (sum(sizes[start_index:end_index]) / sum(sizes)) * 360 / 2 + (start_index / len(sizes) * 360)
    ax.text(np.cos(np.deg2rad(angle)) * (1 + 0.1), np.sin(np.deg2rad(angle)) * (1 + 0.1), f"{cat}\n{sum(sizes[start_index:end_index]):.1f}%", ha='center', va='center')
    start_index = end_index

# 画圆心为空的圆，创造甜甜圈效果
centre_circle = plt.Circle((0,0),0.70,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

# Equal aspect ratio ensures that pie is drawn as a circle.
ax.axis('equal')  
plt.title('Feature Importances by Category')  # 标题

# 添加图例
legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label=label,
                              markerfacecolor=color[0], markersize=10)
                   for cat, label, color in zip(category_dict.keys(), legend_labels, category_colors.values())]
plt.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.15, 1))

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 直接提供特征及其对应的SHAP值
feature_shap_values = [
    ('ASWR', 0.60), ('AAS', 0.51), ('FF', 0.24), ('AMT', 0.15),
    ('EC', 0.15), ('GF', 0.14), ('GDP', 0.13), ('CF', 0.12),
    ('AMMTmax', 0.10), ('NLD', 0.08), ('BLF', 0.08), ('RD', 0.08),
    ('FAR', 0.05), ('BCM', 0.05), ('NDVI', 0.04), ('AF', 0.04),
    ('TSF', 0.04), ('WBF', 0.03), ('MULT', 0.03), ('RF', 0.02),
    ('PGF', 0.02), ('SF', 0.00)
]

# 计算SHAP值总和
total_shap_value = sum(value for _, value in feature_shap_values)

# 分类字典
category_dict = {
    'BE': ['ASWR', 'AAS', 'FF', 'AMT', 'GF'],
    'SC': ['EC', 'GDP', 'CF', 'AMMTmax', 'BLF'],
    'LU': ['RD', 'NLD', 'FAR', 'WBF', 'NDVI'],
    'BC': ['AF', 'PGF', 'RF', 'TSF', 'BCM'],
    'SS': ['MULT', 'SF']
}

# 根据类别给特征分组并生成对应的颜色
category_colors = {
    'BE': plt.cm.Reds(np.linspace(0.4, 1, len(category_dict['BE']))),
    'SC': plt.cm.Blues(np.linspace(0.4, 1, len(category_dict['SC']))),
    'LU': plt.cm.Greens(np.linspace(0.4, 1, len(category_dict['LU']))),
    'BC': plt.cm.Purples(np.linspace(0.4, 1, len(category_dict['BC']))),
    'SS': plt.cm.Greys(np.linspace(0.4, 1, len(category_dict['SS'])))
}

# 创建所有类别的大小和颜色
sizes = []
colors = []
labels = []

for cat, features in category_dict.items():
    for feature in features:
        shap_value = next(val for name, val in feature_shap_values if name == feature)
        sizes.append(shap_value / total_shap_value)  # 使用比例而非原始值
        colors.append(category_colors[cat][features.index(feature)])
        labels.append(f"{feature}: {shap_value / total_shap_value * 100:.1f}%")

# 创建甜甜圈图
fig, ax = plt.subplots(figsize=(10, 7), dpi=1000)

# 绘制细化的甜甜圈图
wedges, texts = ax.pie(sizes, labels=None, autopct=None, startangle=90,
                       wedgeprops=dict(width=0.3, edgecolor='none'), colors=colors)

# 外部标签仅显示主要类别
legend_labels = []
start_index = 0
for cat, features in category_dict.items():
    legend_labels.append(f'{cat} ({", ".join(features)})')
    end_index = start_index + len(features)
    # 设置主类别标签位置
    angle = (sum(sizes[start_index:end_index]) / sum(sizes)) * 360 / 2 + (start_index / len(sizes) * 360)
    ax.text(np.cos(np.deg2rad(angle)) * (1 + 0.1), np.sin(np.deg2rad(angle)) * (1 + 0.1), f"{cat}\n{sum(sizes[start_index:end_index]) * total_shap_value:.1f}", ha='center', va='center')
    start_index = end_index

# 画圆心为空的圆，创造甜甜圈效果
centre_circle = plt.Circle((0,0), 0.50, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

# Equal aspect ratio ensures that pie is drawn as a circle.
ax.axis('equal')  
plt.title('Feature Importances by Category')  # 标题

# 添加图例
legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label=label,
                              markerfacecolor=color, markersize=10)
                   for label, color in zip(labels, colors)]

# 调整图例布局为五列
ax.legend(handles=legend_elements, ncol=5, loc='upper center', bbox_to_anchor=(0.5, -0.1))

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 直接提供特征及其对应的SHAP值
feature_shap_values = [
    ('ASWR', 0.60), ('AAS', 0.51), ('FF', 0.24), ('AMT', 0.15),
    ('EC', 0.15), ('GF', 0.14), ('GDP', 0.13), ('CF', 0.12),
    ('AMMTmax', 0.10), ('NLD', 0.08), ('BLF', 0.08), ('RD', 0.08),
    ('FAR', 0.05), ('BCM', 0.05), ('NDVI', 0.04), ('AF', 0.04),
    ('TSF', 0.04), ('WBF', 0.03), ('MULT', 0.03), ('RF', 0.02),
    ('PGF', 0.02), ('SF', 0.00)
]

# 计算SHAP值总和
total_shap_value = sum(value for _, value in feature_shap_values)

# 分类字典
category_dict = {
    'BE': ['ASWR', 'AAS', 'FF', 'AMT', 'GF'],
    'SC': ['EC', 'GDP', 'CF', 'AMMTmax', 'BLF'],
    'LU': ['RD', 'NLD', 'FAR', 'WBF', 'NDVI'],
    'BC': ['AF', 'PGF', 'RF', 'TSF', 'BCM'],
    'SS': ['MULT', 'SF']
}

# 创建颜色映射
category_colormaps = {'BE': 'Reds', 'SC': 'Blues', 'LU': 'Greens', 'BC': 'Purples', 'SS': 'Greys'}
category_colors = {cat: plt.cm.get_cmap(cmap)(np.linspace(0, 1, len(features))) for cat, (cname, features) in zip(category_colormaps.keys(), category_dict.items()) for cmap in [category_colormaps[cat]]}

sizes = []
colors = []
labels = []

for cat, features in category_dict.items():
    # 根据类别给特征分组并生成对应的颜色
    color_list = category_colors[cat]
    
    for i, feature in enumerate(features):
        shap_value = next(val for name, val in feature_shap_values if name == feature)
        sizes.append(shap_value / total_shap_value)  # 使用比例而非原始值
        colors.append(color_list[i])
        labels.append(f"{feature}: {shap_value / total_shap_value * 100:.1f}%")

# 创建甜甜圈图
fig, ax = plt.subplots(figsize=(10, 7), dpi=100)

# 绘制细化的甜甜圈图
wedges, texts = ax.pie(sizes, labels=None, autopct=None, startangle=90,
                       wedgeprops=dict(width=0.3, edgecolor='none'), colors=colors)

# 外部标签仅显示主要类别
start_index = 0
for cat, features in category_dict.items():
    end_index = start_index + len(features)
    if cat == 'BE':
        be_sum_percentage = sum(sizes[start_index:end_index]) * 100
        angle = (sum(sizes[start_index:end_index]) / sum(sizes)) * 360 / 2 + (start_index / len(sizes) * 360)
        ax.text(np.cos(np.deg2rad(angle)) * (1 + 0.1), np.sin(np.deg2rad(angle)) * (1 + 0.1), f"{cat}\n{be_sum_percentage:.1f}%", ha='center', va='center')
    else:
        angle = (sum(sizes[start_index:end_index]) / sum(sizes)) * 360 / 2 + (start_index / len(sizes) * 360)
        ax.text(np.cos(np.deg2rad(angle)) * (1 + 0.1), np.sin(np.deg2rad(angle)) * (1 + 0.1), f"{cat}\n{sum(sizes[start_index:end_index]) * total_shap_value:.1f}", ha='center', va='center')
    start_index = end_index

# 画圆心为空的圆，创造甜甜圈效果
centre_circle = plt.Circle((0,0), 0.50, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

# Equal aspect ratio ensures that pie is drawn as a circle.
ax.axis('equal')  
plt.title('Feature Importances by Category')  # 标题

# 添加图例
legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label=label,
                              markerfacecolor=color, markersize=10)
                   for label, color in zip(labels, colors)]

# 调整图例布局为五列
ax.legend(handles=legend_elements, ncol=5, loc='upper center', bbox_to_anchor=(0.5, -0.1))

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 直接提供特征及其对应的SHAP值
feature_shap_values = [
    ('ASWR', 0.60), ('AAS', 0.51), ('FF', 0.24), ('AMT', 0.15),
    ('EC', 0.15), ('GF', 0.14), ('GDP', 0.13), ('CF', 0.12),
    ('AMMTmax', 0.10), ('NLD', 0.08), ('BLF', 0.08), ('RD', 0.08),
    ('FAR', 0.05), ('BCM', 0.05), ('NDVI', 0.04), ('AF', 0.04),
    ('TSF', 0.04), ('WBF', 0.03), ('MULT', 0.03), ('RF', 0.02),
    ('PGF', 0.02), ('SF', 0.00)
]

# 计算SHAP值总和
total_shap_value = sum(value for _, value in feature_shap_values)

# 分类字典
category_dict = {
    'BE': ['RD', 'FAR', 'NDVI', 'MULT'],
    'SE': ['EC', 'GDP', 'NLD'],
    'LU': ['FF', 'GF', 'CF', 'BLF', 'AF', 'TSF', 'WBF', 'RF', 'PGF', 'SF'],
    'ML': ['ASWR', 'AAS', 'AMT', 'AMMTmax'],
    'UM': ['RD', 'FAR', 'NDVI', 'MULT']
}

# 根据类别给特征分组并生成对应的颜色
category_colors = {
    'BE': plt.cm.Reds(np.linspace(0.4, 1, len(category_dict['BE']))),
    'SE': plt.cm.Blues(np.linspace(0.4, 1, len(category_dict['SE']))),
    'LU': plt.cm.Greens(np.linspace(0.4, 1, len(category_dict['LU']))),
    'ML': plt.cm.Purples(np.linspace(0.4, 1, len(category_dict['ML']))),
    'UM': plt.cm.Greys(np.linspace(0.4, 1, len(category_dict['UM'])))
}

# 创建所有类别的大小和颜色
sizes = []
colors = []

for cat, features in category_dict.items():
    for feature in features:
        shap_value = next(val for name, val in feature_shap_values if name == feature)
        sizes.append(shap_value / total_shap_value)  # 使用比例而非原始值
        colors.append(category_colors[cat][features.index(feature)])

# 创建甜甜圈图
fig, ax = plt.subplots(figsize=(10, 7))

# 绘制细化的甜甜圈图
wedges, texts = ax.pie(sizes, labels=None, autopct=None, startangle=90,
                       wedgeprops=dict(width=0.3, edgecolor='none'), colors=colors)

# 画圆心为空的圆，创造甜甜圈效果
centre_circle = plt.Circle((0,0), 0.5, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

# 添加总SHAP值到白色圆心内
ax.text(0, 0, f"Total SHAP = {total_shap_value:.3f}", ha='center', va='center', fontsize=12)

# Equal aspect ratio ensures that pie is drawn as a circle.
ax.axis('equal')  
plt.title('Feature Importances by Category')  # 标题
# 添加图例
legend_elements = []
for cat, features in category_dict.items():
    legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', label=f'{cat}', markersize=10))
    for i, feature in enumerate(features):
        # 确保颜色索引不会超出范围
        color_index = min(i, len(category_colors[cat])-1)
        legend_elements.append(plt.Line2D([0], [0], marker='s', color='w', label=f'  {feature}: {(next(val for name, val in feature_shap_values if name == feature) / total_shap)*100:.2f}%', markerfacecolor=category_colors[cat][color_index], markersize=10))

# 显示图例，位置在图形下方，且按大类分为五部分显示
plt.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=5, frameon=False, handletextpad=0.5, handlelength=1)

plt.tight_layout()
plt.show()

# 移除外部标签的代码部分被省略

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colormaps
import numpy as np

# 示例数据：特征及其对应的SHAP值
feature_shap_values = [
    ('ASWR', 8.92), ('AAS', 5.58), ('FF', 3.72), ('AMT', 2.97), ('GF', 1.12),
    ('EC', 8.18), ('GDP', 7.06), ('CF', 5.95), ('AMMTmax', 4.09), ('BLF', 2.60),
    ('RD', 7.43), ('NLD', 6.32), ('FAR', 5.20), ('WBF', 4.09), ('NDVI', 2.97),
    ('AF', 6.69), ('PGF', 5.58), ('RF', 4.46), ('TSF', 3.35), ('BCM', 2.23),
    ('MULT', 1.12), ('SF', 0.37)
]

# 分类字典
category_dict = {
    'BE': ['ASWR', 'AAS', 'FF', 'AMT', 'GF'],
    'SC': ['EC', 'GDP', 'CF', 'AMMTmax', 'BLF'],
    'LU': ['RD', 'NLD', 'FAR', 'WBF', 'NDVI'],
    'BC': ['AF', 'PGF', 'RF', 'TSF', 'BCM'],
    'SS': ['MULT', 'SF']
}

# 计算总SHAP值
total_shap = sum([val for _, val in feature_shap_values])

# 创建颜色映射
category_colormaps = {'BE': 'Reds', 'SC': 'Blues', 'LU': 'Greens', 'BC': 'Purples', 'SS': 'Greys'}
category_colors = {cat: colormaps[cm_name](np.linspace(0, 1, len(category_dict[cat]))) for cat, cm_name in category_colormaps.items()}

# 绘制环形图
fig, ax = plt.subplots(figsize=(10, 7))

# 计算每个变量的占比
sizes = [shap_value / total_shap for _, shap_value in feature_shap_values]

# 获取所有颜色
colors = [category_colors[cat][i] for cat, features in category_dict.items() for i, feature in enumerate(features)]

# 绘制环形图（注意：pie函数返回的是wedges和texts两个元素）
wedges, texts = ax.pie(sizes, colors=colors, startangle=90, 
                       wedgeprops=dict(width=0.3, edgecolor='none'), radius=1)

# 确保在绘制完所有其他图形元素后添加centre_circle
centre_circle = plt.Circle((0,0), 0.2, fc='white')  # 调整这里的半径值来改变空心圆的大小

# 添加centre_circle到当前图形，并确保它位于饼图之上
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

# Equal aspect ratio ensures that pie is drawn as a circle.
ax.axis('equal')
plt.title('Feature Importances by Category')

# 添加图例
legend_elements = []
for cat, features in category_dict.items():
    legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', label=f'{cat}', markersize=10))
    for i, feature in enumerate(features):
        # 确保颜色索引不会超出范围
        color_index = min(i, len(category_colors[cat])-1)
        legend_elements.append(plt.Line2D([0], [0], marker='s', color='w', label=f'  {feature}: {(next(val for name, val in feature_shap_values if name == feature) / total_shap)*100:.2f}%', markerfacecolor=category_colors[cat][color_index], markersize=10))

# 显示图例，位置在图形下方，且按大类分为五部分显示
plt.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=5, frameon=False, handletextpad=0.5, handlelength=1)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 直接提供特征及其对应的SHAP值
feature_shap_values = [
    ('ASWR', 0.60), ('AAS', 0.51), ('FF', 0.24), ('AMT', 0.15),
    ('EC', 0.15), ('GF', 0.14), ('GDP', 0.13), ('CF', 0.12),
    ('AMMTmax', 0.10), ('NLD', 0.08), ('BLF', 0.08), ('RD', 0.08),
    ('FAR', 0.05), ('BCM', 0.05), ('NDVI', 0.04), ('AF', 0.04),
    ('TSF', 0.04), ('WBF', 0.03), ('MULT', 0.03), ('RF', 0.02),
    ('PGF', 0.02), ('SF', 0.00)
]

# 计算SHAP值总和
total_shap_value = sum(value for _, value in feature_shap_values)

# 分类字典
category_dict = {
    'BE': ['RD', 'FAR', 'NDVI', 'MULT'],
    'SE': ['EC', 'GDP', 'NLD'],
    'LU': ['FF', 'GF', 'CF', 'BLF', 'AF', 'TSF', 'WBF', 'RF', 'PGF', 'SF'],
    'ML': ['ASWR', 'AAS', 'AMT', 'AMMTmax'],
    'UM': ['RD', 'FAR', 'NDVI', 'MULT']
}

# 根据类别给特征分组并生成对应的颜色
category_colors = {
    'BE': plt.cm.Reds(np.linspace(0.4, 1, len(category_dict['BE']))),
    'SE': plt.cm.Blues(np.linspace(0.4, 1, len(category_dict['SE']))),
    'LU': plt.cm.Greens(np.linspace(0.4, 1, len(category_dict['LU']))),
    'ML': plt.cm.Purples(np.linspace(0.4, 1, len(category_dict['ML']))),
    'UM': plt.cm.Greys(np.linspace(0.4, 1, len(category_dict['UM'])))
}

fig, ax = plt.subplots(figsize=(10, 8))

y_positions = np.arange(len(category_dict))  # 每个大类别的位置
bar_height = 0.45  # 条形图的高度

for idx, (cat, features) in enumerate(category_dict.items()):
    sizes = [next(val for name, val in feature_shap_values if name == feature) / total_shap_value for feature in features]
    colors = category_colors[cat][:len(features)]
    
    left = 0
    for i, size in enumerate(sizes):
        ax.barh(y_positions[idx], size, height=bar_height, left=left, color=colors[i])
        left += size
    
    # 计算并显示每个大类别的百分比
    cat_total = sum(sizes)
    ax.text(left + 0.00, y_positions[idx], f'{cat_total*100:.1f}%', va='center', ha='left', color='black', fontsize=15)
    
    # 添加类别名称
    ax.text(-0.01, y_positions[idx], cat, va='center', ha='right')

ax.invert_yaxis()  # 反转y轴，使BE在最上方

plt.title('Feature Importances by Category')
plt.tight_layout()

# 增加网格线
ax.grid(True, axis='x', linestyle='--', which='major', color='gray', alpha=0.7)
ax.set_axisbelow(True)  # 网格线置于条形图之下

# 添加图例
legend_elements = []
for cat, features in category_dict.items():
    legend_elements.append(plt.Line2D([0], [0], marker='s', color='w', label=f'{cat}', markersize=6))
    for i, feature in enumerate(features):
        color_index = min(i, len(category_colors[cat])-1)
        legend_elements.append(plt.Line2D([0], [0], marker='s', color='w', label=f'  {feature}: {(next(val for name, val in feature_shap_values if name == feature) / total_shap_value)*100:.2f}%', markerfacecolor=category_colors[cat][color_index], markersize=10))

# 将图例移动到条形图的右侧
plt.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(1.12, 1.02), frameon=False, handletextpad=0.5, handlelength=1)

plt.show()

In [ ]:
#shap.summary_plot(shap_values, data[cols], plot_type="bar")

In [ ]:
shap_interaction_values = shap.TreeExplainer(model).shap_interaction_values(data[cols])

In [ ]:
shap.summary_plot(shap_interaction_values, data[cols], max_display=12)

In [ ]:
plt.rcParams['font.size'] = 14
plt.rcParams['figure.dpi'] = 1000
plt.rcParams['font.family'] = 'Times New Roman'

In [ ]:
#正确修改了横坐标标签科学计数法
import os
import shap
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

# 定义要分析的变量组合
variable_pairs = [
    ('FAR', 'NDVI'), ('FAR', 'EC'), ('FAR', 'GDP'), ('FAR', 'SF'), ('FAR', 'FF'),
    ('FAR', 'ASWR'), ('FAR', 'AAS'), ('FAR', 'PD'), ('FAR', 'DGF'),
    ('NDVI', 'EC'), ('NDVI', 'GDP'), ('NDVI', 'SF'), ('NDVI', 'FF'),
    ('NDVI', 'ASWR'), ('NDVI', 'AAS'), ('NDVI', 'PD'), ('NDVI', 'DGF'),
    ('EC', 'GDP'), ('EC', 'SF'), ('EC', 'FF'), ('EC', 'ASWR'), ('EC', 'AAS'),
    ('EC', 'PD'), ('EC', 'DGF'), ('GDP', 'SF'), ('GDP', 'FF'), ('GDP', 'ASWR'),
    ('GDP', 'AAS'), ('GDP', 'PD'), ('GDP', 'DGF'), ('SF', 'FF'), ('SF', 'ASWR'),
    ('SF', 'AAS'), ('SF', 'PD'), ('SF', 'DGF'), ('FF', 'ASWR'), ('FF', 'AAS'),
    ('FF', 'PD'), ('FF', 'DGF'), ('ASWR', 'AMT'), ('ASWR', 'PD'),
    ('ASWR', 'DGF'), ('AAS', 'PD'), ('AAS', 'DGF'), ('PD', 'DGF')
]

# 创建文件夹用于保存图像
output_folder = r'F:\第三篇论文写作\代码出图\PDP合并\新交互作用\交互作用-2'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 循环遍历每个变量组合并生成依赖图
for var1, var2 in variable_pairs:
    # 如果第二个变量是None，则只绘制单个变量的影响
    if var2 is None:
        shap.dependence_plot(var1, shap_values, data[cols], show=False, cmap="viridis")
    else:
        shap.dependence_plot(var1, shap_values, data[cols], interaction_index=var2, show=False, cmap="viridis")

    # 获取当前图形和轴
    fig = plt.gcf()
    ax = plt.gca()

    # 设置xy轴标签字体和大小
    ax.set_xlabel(ax.get_xlabel(), fontsize=24, fontname='Times New Roman')
    ax.set_ylabel(ax.get_ylabel(), fontsize=24, fontname='Times New Roman')

    # 设置刻度标签字体和大小
    for tick in ax.get_xticklabels() + ax.get_yticklabels():
        tick.set_fontsize(24)
        tick.set_fontname('Times New Roman')

    # === 强制启用科学计数法（关键修改部分）===
    ax.ticklabel_format(style='sci', axis='both', scilimits=(-3, 3), useOffset=False)

    # 可选：如果你还想进一步控制字体大小和样式
    ax.xaxis.offsetText.set_fontsize(20)
    ax.yaxis.offsetText.set_fontsize(20)
    ax.xaxis.offsetText.set_fontname('Times New Roman')
    ax.yaxis.offsetText.set_fontname('Times New Roman')

    # 保存图像
    plt.savefig(os.path.join(output_folder, f'{var1}_{var2}.png'), bbox_inches='tight', dpi=1200)
    plt.close()  # 关闭当前图像以释放内存

print("所有图像已保存完毕。")

In [ ]:
import os
import shap
import matplotlib.pyplot as plt

# 假设shap_values, data 和 cols 已经定义
# shap_values = ...
# data = ...
# cols = ...

# 定义要分析的变量组合
variable_pairs = [
    ('FAR', 'NDVI'), ('FAR', 'EC'), ('FAR', 'GDP'), ('FAR', 'SF'), ('FAR', 'FF'),
    ('FAR', 'ASWR'), ('FAR', 'AAS'), ('FAR', 'PD'), ('FAR', 'DGF'),
    ('NDVI', 'EC'), ('NDVI', 'GDP'), ('NDVI', 'SF'), ('NDVI', 'FF'),
    ('NDVI', 'ASWR'), ('NDVI', 'AAS'), ('NDVI', 'PD'), ('NDVI', 'DGF'),
    ('EC', 'GDP'), ('EC', 'SF'), ('EC', 'FF'), ('EC', 'ASWR'), ('EC', 'AAS'),
    ('EC', 'PD'), ('EC', 'DGF'), ('GDP', 'SF'), ('GDP', 'FF'), ('GDP', 'ASWR'),
    ('GDP', 'AAS'), ('GDP', 'PD'), ('GDP', 'DGF'), ('SF', 'FF'), ('SF', 'ASWR'),
    ('SF', 'AAS'), ('SF', 'PD'), ('SF', 'DGF'), ('FF', 'ASWR'), ('FF', 'AAS'),
    ('FF', 'PD'), ('FF', 'DGF'), ('ASWR', 'AMT'), ('ASWR', 'PD'),
    ('ASWR', 'DGF'), ('AAS', 'PD'), ('AAS', 'DGF'), ('PD', 'DGF')
]

# 创建文件夹用于保存图像
output_folder = r'F:\第三篇论文写作\代码出图\PDP合并\新交互作用\交互作用-2'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 循环遍历每个变量组合并生成依赖图
for var1, var2 in variable_pairs:
    if var2 is None:  # 单变量情况
        shap.dependence_plot(var1, shap_values, data[cols], show=False, cmap="viridis")
    else:  # 双变量情况
        shap.dependence_plot(var1, shap_values, data[cols], interaction_index=var2, cmap="viridis", show=False)
    
    fig, ax = plt.gcf(), plt.gca()
    
    # 设置xy轴标签字体和大小
    ax.set_xlabel(ax.get_xlabel(), fontsize=24, fontname='Times New Roman')
    ax.set_ylabel(ax.get_ylabel(), fontsize=24, fontname='Times New Roman')
    
    # 设置刻度标签字体和大小
    for tick in ax.get_xticklabels() + ax.get_yticklabels():
        tick.set_fontsize(24)
        tick.set_fontname('Times New Roman')
    
    # 对于包含特定变量（EC, GDP, NDVI）的情况，保持科学计数形式
    if 'EC' in [var1, var2] or 'GDP' in [var1, var2] or 'NDVI' in [var1, var2]or 'SF' in [var1, var2]:
        ax.ticklabel_format(style='sci', scilimits=(-3,4), axis='both')
    
    # 保存图像
    plt.savefig(os.path.join(output_folder, f'{var1}_{var2}.png'), bbox_inches='tight', dpi=1200)
    plt.close()  # 关闭当前图像以释放内存

print("所有图像已保存完毕。")

In [ ]:
# 描绘两个变量交互下变量对目标值的影响，选择用哪两个变量交互
shap.dependence_plot('duanbofushe', shap_values, data[cols], interaction_index='jiangshuiliang', show=False)

In [ ]:
# 描绘两个变量交互下变量对目标值的影响，选择用哪两个变量交互
shap.dependence_plot('duanbofushe', shap_values, data[cols], interaction_index='qiya', show=False)

In [ ]:
# 描绘两个变量交互下变量对目标值的影响，选择用哪两个变量交互
shap.dependence_plot('duanbofushe', shap_values, data[cols], interaction_index='p_sl', show=False)

In [ ]:
# 描绘两个变量交互下变量对目标值的影响，选择用哪两个变量交互
shap.dependence_plot('duanbofushe', shap_values, data[cols], interaction_index='rizhaodanweishu', show=False)

In [ ]:
# 描绘两个变量交互下变量对目标值的影响，选择用哪两个变量交互
shap.dependence_plot('duanbofushe', shap_values, data[cols], interaction_index='shape_coefficient_sum', show=False)

In [ ]:
# 描绘两个变量交互下变量对目标值的影响，选择用哪两个变量交互
shap.dependence_plot('build_md', shap_values, data[cols], interaction_index='tccmd', show=False)

In [ ]:
# 描绘两个变量交互下变量对目标值的影响，选择用哪两个变量交互
shap.dependence_plot('nl', shap_values, data[cols], interaction_index='sl', show=False)

In [ ]:
# 描绘两个变量交互下变量对目标值的影响，选择用哪两个变量交互
shap.dependence_plot('nl', shap_values, data[cols], interaction_index='sd', show=False)

In [ ]:
# 描绘两个变量交互下变量对目标值的影响，选择用哪两个变量交互
shap.dependence_plot('nl', shap_values, data[cols], interaction_index='gyldyd', show=False)

In [ ]:
# 描绘两个变量交互下变量对目标值的影响，选择用哪两个变量交互
shap.dependence_plot('nl', shap_values, data[cols], interaction_index='pop_midu', show=False)

In [ ]:
# 描绘两个变量交互下变量对目标值的影响，选择用哪两个变量交互
shap.dependence_plot('nl', shap_values, data[cols], interaction_index='tccmd', show=False)

In [ ]:
# 描绘两个变量交互下变量对目标值的影响，选择用哪两个变量交互
shap.dependence_plot('sl', shap_values, data[cols], interaction_index='sd', show=False)

In [ ]:
# 描绘两个变量交互下变量对目标值的影响，选择用哪两个变量交互
shap.dependence_plot('sl', shap_values, data[cols], interaction_index='gyldyd', show=False)

In [ ]:
# 描绘两个变量交互下变量对目标值的影响，选择用哪两个变量交互
shap.dependence_plot('sl', shap_values, data[cols], interaction_index='pop_midu', show=False)

In [ ]:
# 描绘两个变量交互下变量对目标值的影响，选择用哪两个变量交互
shap.dependence_plot('sl', shap_values, data[cols], interaction_index='tccmd', show=False)

In [ ]:
# 描绘两个变量交互下变量对目标值的影响，选择用哪两个变量交互
shap.dependence_plot('sd', shap_values, data[cols], interaction_index='gyldyd', show=False)

In [ ]:
# 描绘两个变量交互下变量对目标值的影响，选择用哪两个变量交互
shap.dependence_plot('sd', shap_values, data[cols], interaction_index='pop_midu', show=False)

In [ ]:
# 描绘两个变量交互下变量对目标值的影响，选择用哪两个变量交互
shap.dependence_plot('sd', shap_values, data[cols], interaction_index='tccmd', show=False)

In [ ]:
# 描绘两个变量交互下变量对目标值的影响，选择用哪两个变量交互
shap.dependence_plot('pop_midu', shap_values, data[cols], interaction_index='tccmd', show=False)

# PDP

In [ ]:
from sklearn.inspection import PartialDependenceDisplay
import matplotlib.pyplot as plt

# 选择两个特征绘制2D PDP
features = ['FAR', 'NDVI']

# 使用 contour_kw 参数绘制2D PDP
fig, ax = plt.subplots(figsize=(10, 6), dpi=1200)

# 注意：在 features 参数中直接传递特征名称列表
display = PartialDependenceDisplay.from_estimator(
    model,  # 确保这里是你训练好的模型对象
    X_train,  # 使用你的训练数据集
    features=[features],  # 传递一个包含特征名称列表的列表
    kind='average',  # 显示单个和交互效应
    grid_resolution=50,
    contour_kw={'cmap': 'viridis', 'alpha': 0.8},
    ax=ax
)

# 添加标题
plt.suptitle('2D Partial Dependence Plot for duanbofushe and jiangshuiliang')

# 显示图表
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.inspection import partial_dependence

def plot_3d_pdp(model, X, features, fixed_feature=None, fixed_value=None, grid_resolution=50):
    
    feature_1, feature_2, feature_3 = features
    
    # 如果没有指定固定特征，默认将第三个特征固定
    if fixed_feature is None:
        fixed_feature = feature_3
    # 如果没有指定固定值，则使用该特征的均值
    if fixed_value is None:
        fixed_value = X[fixed_feature].mean()
    
    # 构建网格，针对两个未固定的特征
    if fixed_feature == feature_1:
        varying_features = [feature_2, feature_3]
    elif fixed_feature == feature_2:
        varying_features = [feature_1, feature_3]
    else:
        varying_features = [feature_1, feature_2]
    
    feature_1_vals = np.linspace(X[varying_features[0]].min(), X[varying_features[0]].max(), grid_resolution)
    feature_2_vals = np.linspace(X[varying_features[1]].min(), X[varying_features[1]].max(), grid_resolution)
    
    grid_1, grid_2 = np.meshgrid(feature_1_vals, feature_2_vals)
    
    # 构建网格数据，传递给模型进行预测
    grid_points = np.c_[grid_1.ravel(), grid_2.ravel()]
    X_grid = np.zeros((grid_points.shape[0], X.shape[1]))  # 创建空白矩阵，维度与X相同
    X_grid[:, X.columns.get_loc(varying_features[0])] = grid_points[:, 0]
    X_grid[:, X.columns.get_loc(varying_features[1])] = grid_points[:, 1]
    X_grid[:, X.columns.get_loc(fixed_feature)] = fixed_value  # 固定特征值
    
    # 对网格数据进行预测
    preds = model.predict(X_grid).reshape(grid_1.shape)
    
    # 绘制3D图像
    fig = plt.figure(figsize=(10, 8), dpi=1200)
    ax = fig.add_subplot(111, projection='3d')
    
    # 绘制表面图
    surface = ax.plot_surface(grid_1, grid_2, preds, cmap='viridis', alpha=0.8)
    
    # 添加颜色热度条
    fig.colorbar(surface, ax=ax, shrink=0.5, aspect=10)
    
    ax.set_xlabel(varying_features[0])
    ax.set_ylabel(varying_features[1])
    ax.set_zlabel('Prediction')
    
    plt.title(f'3D Partial Dependence Plot for {varying_features[0]}, {varying_features[1]} with {fixed_feature}={fixed_value}')
    plt.savefig("3D_Partial_Dependence_Plot.pdf", format='pdf', bbox_inches='tight')
    plt.show()

# 示例调用函数
features = ['FAR', 'NDVI', 'EC']
plot_3d_pdp(model, X_test, features, fixed_feature='EC')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.inspection import partial_dependence

def plot_3d_pdp_scatter(model, X, features, grid_resolution=20):
    
    # 获取三个特征的数据
    feature_1, feature_2, feature_3 = features
    
    # 构建网格
    feature_1_vals = np.linspace(X[feature_1].min(), X[feature_1].max(), grid_resolution)
    feature_2_vals = np.linspace(X[feature_2].min(), X[feature_2].max(), grid_resolution)
    feature_3_vals = np.linspace(X[feature_3].min(), X[feature_3].max(), grid_resolution)
    
    # 生成三维网格
    grid_1, grid_2, grid_3 = np.meshgrid(feature_1_vals, feature_2_vals, feature_3_vals)
    
    # 计算部分依赖值
    pdp_results = partial_dependence(
        estimator=model,
        X=X,
        features=[X.columns.get_loc(feature_1), X.columns.get_loc(feature_2), X.columns.get_loc(feature_3)],
        grid_resolution=grid_resolution
    )
    
    preds = pdp_results['average'][0]
    
    # 绘制3D散点图
    fig = plt.figure(figsize=(10, 8), dpi=1200)
    ax = fig.add_subplot(111, projection='3d')
    
    # 使用散点图绘制三维特征，并使用预测值作为颜色
    sc = ax.scatter(grid_1.ravel(), grid_2.ravel(), grid_3.ravel(), c=preds, cmap='viridis', alpha=0.8)
    
    # 添加颜色条
    plt.colorbar(sc, ax=ax, label='Prediction')
    ax.set_xlabel(feature_1)
    ax.set_ylabel(feature_2)
    ax.set_zlabel(feature_3)
    
    plt.title(f'3D Partial Dependence Scatter Plot for {feature_1}, {feature_2}, and {feature_3}')
    plt.savefig("3D_Partial_Dependence_Scatter_Plot.pdf", format='pdf', bbox_inches='tight')
    plt.show()

# 示例调用函数
features = ['FAR', 'NDVI', 'EC']
plot_3d_pdp_scatter(model, X_test, features)

In [ ]:
from pdpbox import pdp
def plot_pdp(model, df, feature, cluster_flag=False, nb_clusters=None, lines_flag=False):
    
    # Create the data that we will plot
    pdp_goals = pdp.pdp_isolate(model=model, dataset=df, model_features=df.columns.tolist(), feature=feature)

    # plot it
    pdp.pdp_plot(pdp_goals, feature, cluster=cluster_flag, n_cluster_centers=nb_clusters, plot_lines=lines_flag,figsize=(20,16))

In [ ]:
from sklearn.inspection import PartialDependenceDisplay
import matplotlib.pyplot as plt

# 选择两个特征绘制2D PDP
features = ['duanbofushe', 'jiangshuiliang']

# 使用 contour_kw 参数绘制2D PDP
fig, ax = plt.subplots(figsize=(10, 6), dpi=1200)

# 注意：在 features 参数中直接传递特征名称列表
display = PartialDependenceDisplay.from_estimator(
    model,  # 确保这里是你训练好的模型对象
    X_train,  # 使用你的训练数据集
    features=[features],  # 传递一个包含特征名称列表的列表
    kind='average',  # 显示单个和交互效应
    grid_resolution=50,
    contour_kw={'cmap': 'viridis', 'alpha': 0.8},
    ax=ax
)

# 添加标题
plt.suptitle('2D Partial Dependence Plot for duanbofushe and jiangshuiliang')

# 显示图表
plt.show()

In [ ]:
from sklearn.inspection import PartialDependenceDisplay
import matplotlib.pyplot as plt

# 选择两个特征绘制2D PDP
features = ['FAR', 'NDVI', 'EC', 'GDP', 'SF', 'FF', 'AMMTmax', 'AMT', 'RD', 'MULT']

# 使用 contour_kw 参数绘制2D PDP
fig, ax = plt.subplots(figsize=(12, 8), dpi=1200)

# 注意：在 features 参数中直接传递特征名称列表
display = PartialDependenceDisplay.from_estimator(
    model,  # 确保这里是你训练好的模型对象
    X_train,  # 使用你的训练数据集
    features=features,  # 直接传递特征名称列表
    kind='both',  # 显示单个和交互效应  交互作用则改为'interaction',两个都出改完'both'单独改为'average'
    grid_resolution=50,
    contour_kw={'cmap': 'viridis', 'alpha': 0.8},
    ax=ax
)

# 添加标题
plt.suptitle('2D Partial Dependence Plot for duanbofushe and jiangshuiliang')

# 调整子图之间的距离
plt.tight_layout()  # 自动调整子图参数,使之填充整个图像区域
plt.subplots_adjust(hspace=0.29, wspace=0.1)  # 调整子图之间的垂直和水平间距

# 显示图表
plt.show()

In [ ]:
#@title Select a predictor to analyse Linear Regression Model. { run: "auto" }

selected_predictor = "duanbofushe" #@param ['Age_08_04', 'KM', 'HP', 'Met_Color', 'Automatic', 'cc', 'Doors', 'Weight']
print('You selected', selected_predictor)
plot_pdp(model, X_train, selected_predictor)

In [ ]:
from pdpbox import pdp
import matplotlib.pyplot as plt

def plot_pdp(model, df, feature, cluster_flag=False, nb_clusters=None, lines_flag=False):
    # Create the data that we will plot
    pdp_goals = pdp.pdp_isolate(model=model, dataset=df, model_features=df.columns.tolist(), feature=feature)

    # Plot it without 'return_fig' parameter
    pdp.pdp_plot(pdp_goals, feature, cluster=cluster_flag, n_cluster_centers=nb_clusters, plot_lines=lines_flag, figsize=(10,8))
    
    # 获取当前的matplotlib figure对象
    fig = plt.gcf()
    ax = plt.gca()
    
    # 修改标签字体大小和字体为新罗马
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(18)  # 设置字体大小
        item.set_fontname('Times New Roman')  # 设置字体为新罗马
    
    # 如果存在图例，则也需要修改其字体
    if ax.get_legend() is not None:
        plt.setp(ax.get_legend().get_texts(), fontsize='14')  # 设置图例字体大小
        plt.setp(ax.get_legend().get_texts(), fontname='Times New Roman')  # 设置图例字体为新罗马

    # 保存高质量的图像文件
    plt.savefig('output.png', dpi=1000)
    
    # 显示图像
    plt.show()

In [ ]:
selected_predictor = 'FAR'
print('You selected', selected_predictor)
plot_pdp(model, X, selected_predictor)

In [ ]:
from pdpbox import pdp
import matplotlib.pyplot as plt

def plot_pdp(model, df, feature, cluster_flag=False, nb_clusters=None, lines_flag=False):
    # Create the data that we will plot
    pdp_goals = pdp.pdp_isolate(model=model, dataset=df, model_features=df.columns.tolist(), feature=feature)

    # Plot it without 'return_fig' parameter
    fig, ax = pdp.pdp_plot(pdp_goals, feature, cluster=cluster_flag, n_cluster_centers=nb_clusters, plot_lines=lines_flag, figsize=(10, 8))
    
    # 获取当前的matplotlib figure对象
    fig = plt.gcf()
    ax = plt.gca()
    
    # 修改标签字体大小和字体为新罗马
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(18)  # 设置字体大小
        item.set_fontname('Times New Roman')  # 设置字体为新罗马
    
    # 如果存在图例，则也需要修改其字体
    if ax.get_legend() is not None:
        plt.setp(ax.get_legend().get_texts(), fontsize='14')  # 设置图例字体大小
        plt.setp(ax.get_legend().get_texts(), fontname='Times New Roman')  # 设置图例字体为新罗马

    # 保存高质量的图像文件
    plt.savefig('output.png', dpi=1000)
    
    # 显示图像
    plt.show()

    # 提取并打印坐标点
    print("Extracting coordinates of the lines...")
    
    # 获取中间线的坐标点
    mid_line_x = ax.lines[0].get_xdata()
    mid_line_y = ax.lines[0].get_ydata()
    print(f"Middle Line Coordinates: {list(zip(mid_line_x, mid_line_y))}")
    
    # 获取上界线的坐标点
    upper_bound_x = ax.collections[0].get_paths()[0].vertices[:, 0]
    upper_bound_y = ax.collections[0].get_paths()[0].vertices[:, 1]
    print(f"Upper Bound Coordinates: {list(zip(upper_bound_x, upper_bound_y))}")
    
    # 获取下界线的坐标点
    lower_bound_x = ax.collections[1].get_paths()[0].vertices[:, 0]
    lower_bound_y = ax.collections[1].get_paths()[0].vertices[:, 1]
    print(f"Lower Bound Coordinates: {list(zip(lower_bound_x, lower_bound_y))}")

selected_predictor = 'FAR'
print('You selected', selected_predictor)
plot_pdp(model, X, selected_predictor)

In [ ]:
from pdpbox import pdp
import matplotlib.pyplot as plt

def plot_pdp(model, df, feature, cluster_flag=False, nb_clusters=None, lines_flag=False):
    # Create the data that we will plot
    pdp_goals = pdp.pdp_isolate(model=model, dataset=df, model_features=df.columns.tolist(), feature=feature)

    # Plot it without 'return_fig' parameter
    fig, ax = pdp.pdp_plot(pdp_goals, feature, cluster=cluster_flag, n_cluster_centers=nb_clusters, plot_lines=lines_flag, figsize=(10, 8))
    
    # 获取当前的matplotlib figure对象
    fig = plt.gcf()
    ax = plt.gca()
    
    # 修改标签字体大小和字体为新罗马
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(18)  # 设置字体大小
        item.set_fontname('Times New Roman')  # 设置字体为新罗马
    
    # 如果存在图例，则也需要修改其字体
    if ax.get_legend() is not None:
        plt.setp(ax.get_legend().get_texts(), fontsize='14')  # 设置图例字体大小
        plt.setp(ax.get_legend().get_texts(), fontname='Times New Roman')  # 设置图例字体为新罗马

    # 保存高质量的图像文件
    plt.savefig('output.png', dpi=1000)
    
    # 显示图像
    plt.show()

    # 提取并打印坐标点
    print("Extracting coordinates of the lines...")
    
    # 获取中间线的坐标点
    mid_line_x = ax.lines[0].get_xdata()
    mid_line_y = ax.lines[0].get_ydata()
    print(f"Middle Line Coordinates: {list(zip(mid_line_x, mid_line_y))}")
    
    # 检查是否有上下界线，并获取它们的坐标点
    if len(ax.collections) > 0:
        upper_bound_path = ax.collections[0].get_paths()[0]
        upper_bound_x = upper_bound_path.vertices[:, 0]
        upper_bound_y = upper_bound_path.vertices[:, 1]
        print(f"Upper Bound Coordinates: {list(zip(upper_bound_x, upper_bound_y))}")
        
        if len(ax.collections) > 1:
            lower_bound_path = ax.collections[1].get_paths()[0]
            lower_bound_x = lower_bound_path.vertices[:, 0]
            lower_bound_y = lower_bound_path.vertices[:, 1]
            print(f"Lower Bound Coordinates: {list(zip(lower_bound_x, lower_bound_y))}")
        else:
            print("No Lower Bound Line detected.")
    else:
        print("No Upper or Lower Bound Lines detected.")

selected_predictor = 'FAR'
print('You selected', selected_predictor)
plot_pdp(model, X, selected_predictor)

In [ ]:
from pdpbox import pdp
import matplotlib.pyplot as plt
import pandas as pd

def print_pdp_values(pdp_result):
    # 获取x轴（特征值）和y轴（偏依赖值）的数据
    feature_values = pdp_result.feature_grids  # 特征的值范围
    pdp_values = pdp_result.pdp  # 平均线的y值
    pdp_std = pdp_result.ice_lines.std(axis=0)  # 计算每个特征值对应的标准差
    
    # 打印中间平均线的坐标点值
    print("Average line coordinates:")
    for x, y in zip(feature_values, pdp_values):
        print(f"Feature value: {x}, PDP value: {y}")
    
    # 打印上下界线的坐标点值
    print("\nConfidence interval boundaries:")
    for x, y, std in zip(feature_values, pdp_values, pdp_std):
        lower_bound = y - 1.96 * std  # 假设95%置信区间
        upper_bound = y + 1.96 * std  # 假设95%置信区间
        print(f"Feature value: {x}, Lower bound: {lower_bound}, Upper bound: {upper_bound}")

def plot_pdp(model, df, feature, cluster_flag=False, nb_clusters=None, lines_flag=False):
    # Create the data that we will plot
    pdp_goals = pdp.pdp_isolate(model=model, dataset=df, model_features=df.columns.tolist(), feature=feature)
    
    # 调用函数打印坐标点值
    print_pdp_values(pdp_goals)

    # Plot it without 'return_fig' parameter
    pdp.pdp_plot(pdp_goals, feature, cluster=cluster_flag, n_cluster_centers=nb_clusters, plot_lines=lines_flag, figsize=(10,8))
    
    # 继续绘制图形...
    fig = plt.gcf()
    ax = plt.gca()
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(18)
        item.set_fontname('Times New Roman')
    if ax.get_legend() is not None:
        plt.setp(ax.get_legend().get_texts(), fontsize='14')
        plt.setp(ax.get_legend().get_texts(), fontname='Times New Roman')
    plt.savefig('output.png', dpi=1000)
    plt.show()

columns = ['FAR']  # 示例中仅包含一个特征，根据实际情况修改

# 确保X是DataFrame类型，如果不是，请转换它
if not isinstance(X, pd.DataFrame):
    X = pd.DataFrame(X, columns=columns)  # 根据实际情况填写列名

selected_predictor = 'FAR'
print('You selected', selected_predictor)

# 确保这里的model和X是你之前使用的数据和模型
plot_pdp(model, X, selected_predictor)

In [ ]:
from pdpbox import pdp
import matplotlib.pyplot as plt
import pandas as pd

def plot_pdp_and_calculate_bounds(model, df, feature, cluster_flag=False, nb_clusters=None, lines_flag=False):
    # Create the data that we will plot
    pdp_goals = pdp.pdp_isolate(model=model, dataset=df, model_features=df.columns.tolist(), feature=feature)

    # Plot it without 'return_fig' parameter
    fig, ax = pdp.pdp_plot(pdp_goals, feature, cluster=cluster_flag, n_cluster_centers=nb_clusters, plot_lines=lines_flag, figsize=(10, 8))
    
    # 获取当前的matplotlib figure对象
    fig = plt.gcf()
    ax = plt.gca()
    
    # 修改标签字体大小和字体为新罗马
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(18)  # 设置字体大小
        item.set_fontname('Times New Roman')  # 设置字体为新罗马
    
    # 如果存在图例，则也需要修改其字体
    if ax.get_legend() is not None:
        plt.setp(ax.get_legend().get_texts(), fontsize='14')  # 设置图例字体大小
        plt.setp(ax.get_legend().get_texts(), fontname='Times New Roman')  # 设置图例字体为新罗马

    # 保存高质量的图像文件
    plt.savefig('output.png', dpi=1000)
    
    # 显示图像
    plt.show()

    # 提取坐标点
    print("Extracting coordinates of the lines...")
    
    # 获取中间线的坐标点
    mid_line_x = ax.lines[0].get_xdata()
    mid_line_y = ax.lines[0].get_ydata()
    print(f"Middle Line Coordinates: {list(zip(mid_line_x, mid_line_y))}")
    
    upper_bound_x, upper_bound_y = [], []
    lower_bound_y = []  # 初始化下界y坐标列表
    
    if len(ax.collections) > 0:
        upper_bound_path = ax.collections[0].get_paths()[0]
        upper_bound_x = upper_bound_path.vertices[:, 0]
        upper_bound_y = upper_bound_path.vertices[:, 1]
        print(f"Upper Bound Coordinates: {list(zip(upper_bound_x, upper_bound_y))}")
        
        # 计算下界
        for i in range(len(mid_line_x)):
            corresponding_upper_y = upper_bound_y[i] if i < len(upper_bound_y) else None
            if corresponding_upper_y is not None:
                lower_bound_y.append(2 * mid_line_y[i] - corresponding_upper_y)
            else:
                lower_bound_y.append(None)
                
        print(f"Calculated Lower Bound Coordinates (x same as Middle Line): {list(zip(mid_line_x, lower_bound_y))}")

        # 将结果保存到CSV文件
        results_df = pd.DataFrame({
            'Feature_Value': mid_line_x,
            'Middle_Line': mid_line_y,
            'Upper_Bound': upper_bound_y[:len(mid_line_x)],  # 确保长度一致
            'Calculated_Lower_Bound': lower_bound_y
        })
        results_df.to_csv(r'F:\第三篇论文写作\代码出图\PDP合并\FAR2.csv', index=False)
        print("Results saved to confidence_intervals.csv")
    else:
        print("No Upper or Lower Bound Lines detected.")

# 假设model和X已经定义
selected_predictor = 'FAR'
print('You selected', selected_predictor)
plot_pdp_and_calculate_bounds(model, X, selected_predictor)

In [ ]:
import pandas as pd
from pdpbox import pdp
import matplotlib.pyplot as plt

def save_pdp_values_to_csv(pdp_result, filename):
    # 获取x轴（特征值）和y轴（偏依赖值）的数据
    feature_values = pdp_result.feature_grids  # 特征的值范围
    pdp_values = pdp_result.pdp  # 平均线的y值
    pdp_std = pdp_result.ice_lines.std(axis=0)  # 计算每个特征值对应的标准差
    
    # 创建DataFrame来存储数据
    data = {
        'Feature Value': feature_values,
        'PDP Value': pdp_values,
        'Lower Bound': [y - 1.96 * std for y, std in zip(pdp_values, pdp_std)],
        'Upper Bound': [y + 1.96 * std for y, std in zip(pdp_values, pdp_std)]
    }
    
    df = pd.DataFrame(data)
    
    # 将DataFrame保存为CSV文件
    df.to_csv(filename, index=False)

def plot_pdp(model, df, feature, cluster_flag=False, nb_clusters=None, lines_flag=False):
    # Create the data that we will plot
    pdp_goals = pdp.pdp_isolate(model=model, dataset=df, model_features=df.columns.tolist(), feature=feature)
    
    # 调用函数保存坐标点值到csv
    save_pdp_values_to_csv(pdp_goals, r'F:\第三篇论文写作\代码出图\PDP合并\FAR2.csv')

    # Plot it without 'return_fig' parameter
    pdp.pdp_plot(pdp_goals, feature, cluster=cluster_flag, n_cluster_centers=nb_clusters, plot_lines=lines_flag, figsize=(10,8))
    
    # 继续绘制图形...
    fig = plt.gcf()
    ax = plt.gca()
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(18)
        item.set_fontname('Times New Roman')
    if ax.get_legend() is not None:
        plt.setp(ax.get_legend().get_texts(), fontsize='14')
        plt.setp(ax.get_legend().get_texts(), fontname='Times New Roman')
    plt.savefig('output.png', dpi=1000)
    plt.show()

# 示例中的columns列表仅包含一个特征，根据实际情况调整
columns = ['FAR']  

# 确保X是DataFrame类型，如果不是，请转换它
if not isinstance(X, pd.DataFrame):
    X = pd.DataFrame(X, columns=columns)  # 根据实际情况填写列名

selected_predictor = 'FAR'
print('You selected', selected_predictor)

# 确保这里的model和X是你之前使用的数据和模型
plot_pdp(model, X, selected_predictor)

In [ ]:
import pandas as pd
from pdpbox import pdp
import matplotlib.pyplot as plt

def plot_pdp(model, df, feature, cluster_flag=False, nb_clusters=None, lines_flag=False):
    # Create the data that we will plot
    pdp_goals = pdp.pdp_isolate(model=model, dataset=df, model_features=df.columns.tolist(), feature=feature)

    # Print the coordinates of the average line and confidence intervals
    print("Average line coordinates:")
    for x, y in zip(pdp_goals.feature_grids, pdp_goals.pdp):
        print(f"Feature value: {x}, PDP value: {y}")
    
    print("\nConfidence interval boundaries:")
    std_devs = pdp_goals.ice_lines.std(axis=0)
    for x, y, std in zip(pdp_goals.feature_grids, pdp_goals.pdp, std_devs):
        lower_bound = y - 1.96 * std  # 95% confidence interval
        upper_bound = y + 1.96 * std  # 95% confidence interval
        print(f"Feature value: {x}, Lower bound: {lower_bound}, Upper bound: {upper_bound}")

    # Plot it without 'return_fig' parameter
    pdp.pdp_plot(pdp_goals, feature, cluster=cluster_flag, n_cluster_centers=nb_clusters, plot_lines=lines_flag, figsize=(10,8))
    
    # 获取当前的matplotlib figure对象
    fig = plt.gcf()
    ax = plt.gca()
    
    # 修改标签字体大小和字体为新罗马
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(18)  # 设置字体大小
        item.set_fontname('Times New Roman')  # 设置字体为新罗马
    
    # 如果存在图例，则也需要修改其字体
    if ax.get_legend() is not None:
        plt.setp(ax.get_legend().get_texts(), fontsize='14')  # 设置图例字体大小
        plt.setp(ax.get_legend().get_texts(), fontname='Times New Roman')  # 设置图例字体为新罗马

    # 保存高质量的图像文件
    plt.savefig('output.png', dpi=1000)
    
    # 显示图像
    plt.show()

# 假设你已经有了数据集 X 和模型 model
# X = ...
# model = ...

selected_predictor = 'FAR'
print('You selected', selected_predictor)
plot_pdp(model, X, selected_predictor)

In [ ]:
selected_predictor = "qiya" 
print('You selected', selected_predictor)
plot_pdp(model, X, selected_predictor)

In [ ]:
selected_predictor = "sd" 
print('You selected', selected_predictor)
plot_pdp(model, X, selected_predictor)

In [ ]:
selected_predictor = "gyldyd" 
print('You selected', selected_predictor)
plot_pdp(model, X, selected_predictor)

In [ ]:
selected_predictor = "pop_midu" 
print('You selected', selected_predictor)
plot_pdp(model, X, selected_predictor)

In [ ]:
selected_predictor = "tccmd" 
print('You selected', selected_predictor)
plot_pdp(model, X, selected_predictor)

In [ ]:
from pdpbox import pdp
import matplotlib.pyplot as plt
import pandas as pd

def plot_pdp_and_calculate_bounds(model, df, feature, cluster_flag=False, nb_clusters=None, lines_flag=False):
    # Create the data that we will plot
    pdp_goals = pdp.pdp_isolate(model=model, dataset=df, model_features=df.columns.tolist(), feature=feature)

    # Plot it without 'return_fig' parameter
    fig, ax = pdp.pdp_plot(pdp_goals, feature, cluster=cluster_flag, n_cluster_centers=nb_clusters, plot_lines=lines_flag, figsize=(10, 8))
    
    # 获取当前的matplotlib figure对象
    fig = plt.gcf()
    ax = plt.gca()
    
    # 修改标签字体大小和字体为新罗马
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(18)  # 设置字体大小
        item.set_fontname('Times New Roman')  # 设置字体为新罗马
    
    # 如果存在图例，则也需要修改其字体
    if ax.get_legend() is not None:
        plt.setp(ax.get_legend().get_texts(), fontsize='14')  # 设置图例字体大小
        plt.setp(ax.get_legend().get_texts(), fontname='Times New Roman')  # 设置图例字体为新罗马

    # 保存高质量的图像文件
    plt.savefig('output.png', dpi=1000)
    
    # 显示图像
    plt.show()

    # 提取坐标点
    print("Extracting coordinates of the lines...")
    
    # 获取中间线的坐标点
    mid_line_x = ax.lines[0].get_xdata()
    mid_line_y = ax.lines[0].get_ydata()
    print(f"Middle Line Coordinates: {list(zip(mid_line_x, mid_line_y))}")
    
    upper_bound_x, upper_bound_y = [], []
    lower_bound_y = []  # 初始化下界y坐标列表
    
    if len(ax.collections) > 0:
        upper_bound_path = ax.collections[0].get_paths()[0]
        upper_bound_x = upper_bound_path.vertices[:, 0]
        upper_bound_y = upper_bound_path.vertices[:, 1]
        print(f"Upper Bound Coordinates: {list(zip(upper_bound_x, upper_bound_y))}")
        
        # 计算下界
        for i in range(len(mid_line_x)):
            corresponding_upper_y = upper_bound_y[i] if i < len(upper_bound_y) else None
            if corresponding_upper_y is not None:
                lower_bound_y.append(2 * mid_line_y[i] - corresponding_upper_y)
            else:
                lower_bound_y.append(None)
                
        print(f"Calculated Lower Bound Coordinates (x same as Middle Line): {list(zip(mid_line_x, lower_bound_y))}")

        # 将结果保存到CSV文件
        results_df = pd.DataFrame({
            'Feature_Value': mid_line_x,
            'Middle_Line': mid_line_y,
            'Upper_Bound': upper_bound_y[:len(mid_line_x)],  # 确保长度一致
            'Calculated_Lower_Bound': lower_bound_y
        })
        results_df.to_csv(r'F:\第三篇论文写作\代码出图\PDP合并\NDVI2.csv', index=False)
        print("Results saved to confidence_intervals.csv")
    else:
        print("No Upper or Lower Bound Lines detected.")

# 假设model和X已经定义
selected_predictor = 'NDVI'
print('You selected', selected_predictor)
plot_pdp_and_calculate_bounds(model, X, selected_predictor)

In [ ]:
from pdpbox import pdp
import matplotlib.pyplot as plt
import pandas as pd

def plot_pdp_and_calculate_bounds(model, df, feature, cluster_flag=False, nb_clusters=None, lines_flag=False):
    # Create the data that we will plot
    pdp_goals = pdp.pdp_isolate(model=model, dataset=df, model_features=df.columns.tolist(), feature=feature)

    # Plot it without 'return_fig' parameter
    fig, ax = pdp.pdp_plot(pdp_goals, feature, cluster=cluster_flag, n_cluster_centers=nb_clusters, plot_lines=lines_flag, figsize=(10, 8))
    
    # 获取当前的matplotlib figure对象
    fig = plt.gcf()
    ax = plt.gca()
    
    # 修改标签字体大小和字体为新罗马
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(18)  # 设置字体大小
        item.set_fontname('Times New Roman')  # 设置字体为新罗马
    
    # 如果存在图例，则也需要修改其字体
    if ax.get_legend() is not None:
        plt.setp(ax.get_legend().get_texts(), fontsize='14')  # 设置图例字体大小
        plt.setp(ax.get_legend().get_texts(), fontname='Times New Roman')  # 设置图例字体为新罗马

    # 保存高质量的图像文件
    plt.savefig('output.png', dpi=1000)
    
    # 显示图像
    plt.show()

    # 提取坐标点
    print("Extracting coordinates of the lines...")
    
    # 获取中间线的坐标点
    mid_line_x = ax.lines[0].get_xdata()
    mid_line_y = ax.lines[0].get_ydata()
    print(f"Middle Line Coordinates: {list(zip(mid_line_x, mid_line_y))}")
    
    upper_bound_x, upper_bound_y = [], []
    lower_bound_y = []  # 初始化下界y坐标列表
    
    if len(ax.collections) > 0:
        upper_bound_path = ax.collections[0].get_paths()[0]
        upper_bound_x = upper_bound_path.vertices[:, 0]
        upper_bound_y = upper_bound_path.vertices[:, 1]
        print(f"Upper Bound Coordinates: {list(zip(upper_bound_x, upper_bound_y))}")
        
        # 计算下界
        for i in range(len(mid_line_x)):
            corresponding_upper_y = upper_bound_y[i] if i < len(upper_bound_y) else None
            if corresponding_upper_y is not None:
                lower_bound_y.append(2 * mid_line_y[i] - corresponding_upper_y)
            else:
                lower_bound_y.append(None)
                
        print(f"Calculated Lower Bound Coordinates (x same as Middle Line): {list(zip(mid_line_x, lower_bound_y))}")

        # 将结果保存到CSV文件
        results_df = pd.DataFrame({
            'Feature_Value': mid_line_x,
            'Middle_Line': mid_line_y,
            'Upper_Bound': upper_bound_y[:len(mid_line_x)],  # 确保长度一致
            'Calculated_Lower_Bound': lower_bound_y
        })
        results_df.to_csv(r'F:\第三篇论文写作\代码出图\PDP合并\EC2.csv', index=False)
        print("Results saved to confidence_intervals.csv")
    else:
        print("No Upper or Lower Bound Lines detected.")

# 假设model和X已经定义
selected_predictor = 'EC'
print('You selected', selected_predictor)
plot_pdp_and_calculate_bounds(model, X, selected_predictor)

In [ ]:
from pdpbox import pdp
import matplotlib.pyplot as plt
import pandas as pd

def plot_pdp_and_calculate_bounds(model, df, feature, cluster_flag=False, nb_clusters=None, lines_flag=False):
    # Create the data that we will plot
    pdp_goals = pdp.pdp_isolate(model=model, dataset=df, model_features=df.columns.tolist(), feature=feature)

    # Plot it without 'return_fig' parameter
    fig, ax = pdp.pdp_plot(pdp_goals, feature, cluster=cluster_flag, n_cluster_centers=nb_clusters, plot_lines=lines_flag, figsize=(10, 8))
    
    # 获取当前的matplotlib figure对象
    fig = plt.gcf()
    ax = plt.gca()
    
    # 修改标签字体大小和字体为新罗马
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(18)  # 设置字体大小
        item.set_fontname('Times New Roman')  # 设置字体为新罗马
    
    # 如果存在图例，则也需要修改其字体
    if ax.get_legend() is not None:
        plt.setp(ax.get_legend().get_texts(), fontsize='14')  # 设置图例字体大小
        plt.setp(ax.get_legend().get_texts(), fontname='Times New Roman')  # 设置图例字体为新罗马

    # 保存高质量的图像文件
    plt.savefig('output.png', dpi=1000)
    
    # 显示图像
    plt.show()

    # 提取坐标点
    print("Extracting coordinates of the lines...")
    
    # 获取中间线的坐标点
    mid_line_x = ax.lines[0].get_xdata()
    mid_line_y = ax.lines[0].get_ydata()
    print(f"Middle Line Coordinates: {list(zip(mid_line_x, mid_line_y))}")
    
    upper_bound_x, upper_bound_y = [], []
    lower_bound_y = []  # 初始化下界y坐标列表
    
    if len(ax.collections) > 0:
        upper_bound_path = ax.collections[0].get_paths()[0]
        upper_bound_x = upper_bound_path.vertices[:, 0]
        upper_bound_y = upper_bound_path.vertices[:, 1]
        print(f"Upper Bound Coordinates: {list(zip(upper_bound_x, upper_bound_y))}")
        
        # 计算下界
        for i in range(len(mid_line_x)):
            corresponding_upper_y = upper_bound_y[i] if i < len(upper_bound_y) else None
            if corresponding_upper_y is not None:
                lower_bound_y.append(2 * mid_line_y[i] - corresponding_upper_y)
            else:
                lower_bound_y.append(None)
                
        print(f"Calculated Lower Bound Coordinates (x same as Middle Line): {list(zip(mid_line_x, lower_bound_y))}")

        # 将结果保存到CSV文件
        results_df = pd.DataFrame({
            'Feature_Value': mid_line_x,
            'Middle_Line': mid_line_y,
            'Upper_Bound': upper_bound_y[:len(mid_line_x)],  # 确保长度一致
            'Calculated_Lower_Bound': lower_bound_y
        })
        results_df.to_csv(r'F:\第三篇论文写作\代码出图\PDP合并\GDP2.csv', index=False)
        print("Results saved to confidence_intervals.csv")
    else:
        print("No Upper or Lower Bound Lines detected.")

# 假设model和X已经定义
selected_predictor = 'GDP'
print('You selected', selected_predictor)
plot_pdp_and_calculate_bounds(model, X, selected_predictor)

In [ ]:
from pdpbox import pdp
import matplotlib.pyplot as plt
import pandas as pd

def plot_pdp_and_calculate_bounds(model, df, feature, cluster_flag=False, nb_clusters=None, lines_flag=False):
    # Create the data that we will plot
    pdp_goals = pdp.pdp_isolate(model=model, dataset=df, model_features=df.columns.tolist(), feature=feature)

    # Plot it without 'return_fig' parameter
    fig, ax = pdp.pdp_plot(pdp_goals, feature, cluster=cluster_flag, n_cluster_centers=nb_clusters, plot_lines=lines_flag, figsize=(10, 8))
    
    # 获取当前的matplotlib figure对象
    fig = plt.gcf()
    ax = plt.gca()
    
    # 修改标签字体大小和字体为新罗马
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(18)  # 设置字体大小
        item.set_fontname('Times New Roman')  # 设置字体为新罗马
    
    # 如果存在图例，则也需要修改其字体
    if ax.get_legend() is not None:
        plt.setp(ax.get_legend().get_texts(), fontsize='14')  # 设置图例字体大小
        plt.setp(ax.get_legend().get_texts(), fontname='Times New Roman')  # 设置图例字体为新罗马

    # 保存高质量的图像文件
    plt.savefig('output.png', dpi=1000)
    
    # 显示图像
    plt.show()

    # 提取坐标点
    print("Extracting coordinates of the lines...")
    
    # 获取中间线的坐标点
    mid_line_x = ax.lines[0].get_xdata()
    mid_line_y = ax.lines[0].get_ydata()
    print(f"Middle Line Coordinates: {list(zip(mid_line_x, mid_line_y))}")
    
    upper_bound_x, upper_bound_y = [], []
    lower_bound_y = []  # 初始化下界y坐标列表
    
    if len(ax.collections) > 0:
        upper_bound_path = ax.collections[0].get_paths()[0]
        upper_bound_x = upper_bound_path.vertices[:, 0]
        upper_bound_y = upper_bound_path.vertices[:, 1]
        print(f"Upper Bound Coordinates: {list(zip(upper_bound_x, upper_bound_y))}")
        
        # 计算下界
        for i in range(len(mid_line_x)):
            corresponding_upper_y = upper_bound_y[i] if i < len(upper_bound_y) else None
            if corresponding_upper_y is not None:
                lower_bound_y.append(2 * mid_line_y[i] - corresponding_upper_y)
            else:
                lower_bound_y.append(None)
                
        print(f"Calculated Lower Bound Coordinates (x same as Middle Line): {list(zip(mid_line_x, lower_bound_y))}")

        # 将结果保存到CSV文件
        results_df = pd.DataFrame({
            'Feature_Value': mid_line_x,
            'Middle_Line': mid_line_y,
            'Upper_Bound': upper_bound_y[:len(mid_line_x)],  # 确保长度一致
            'Calculated_Lower_Bound': lower_bound_y
        })
        results_df.to_csv(r'F:\第三篇论文写作\代码出图\PDP合并\SF2.csv', index=False)
        print("Results saved to confidence_intervals.csv")
    else:
        print("No Upper or Lower Bound Lines detected.")

# 假设model和X已经定义
selected_predictor = 'SF'
print('You selected', selected_predictor)
plot_pdp_and_calculate_bounds(model, X, selected_predictor)

In [ ]:
from pdpbox import pdp
import matplotlib.pyplot as plt
import pandas as pd

def plot_pdp_and_calculate_bounds(model, df, feature, cluster_flag=False, nb_clusters=None, lines_flag=False):
    # Create the data that we will plot
    pdp_goals = pdp.pdp_isolate(model=model, dataset=df, model_features=df.columns.tolist(), feature=feature)

    # Plot it without 'return_fig' parameter
    fig, ax = pdp.pdp_plot(pdp_goals, feature, cluster=cluster_flag, n_cluster_centers=nb_clusters, plot_lines=lines_flag, figsize=(10, 8))
    
    # 获取当前的matplotlib figure对象
    fig = plt.gcf()
    ax = plt.gca()
    
    # 修改标签字体大小和字体为新罗马
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(18)  # 设置字体大小
        item.set_fontname('Times New Roman')  # 设置字体为新罗马
    
    # 如果存在图例，则也需要修改其字体
    if ax.get_legend() is not None:
        plt.setp(ax.get_legend().get_texts(), fontsize='14')  # 设置图例字体大小
        plt.setp(ax.get_legend().get_texts(), fontname='Times New Roman')  # 设置图例字体为新罗马

    # 保存高质量的图像文件
    plt.savefig('output.png', dpi=1000)
    
    # 显示图像
    plt.show()

    # 提取坐标点
    print("Extracting coordinates of the lines...")
    
    # 获取中间线的坐标点
    mid_line_x = ax.lines[0].get_xdata()
    mid_line_y = ax.lines[0].get_ydata()
    print(f"Middle Line Coordinates: {list(zip(mid_line_x, mid_line_y))}")
    
    upper_bound_x, upper_bound_y = [], []
    lower_bound_y = []  # 初始化下界y坐标列表
    
    if len(ax.collections) > 0:
        upper_bound_path = ax.collections[0].get_paths()[0]
        upper_bound_x = upper_bound_path.vertices[:, 0]
        upper_bound_y = upper_bound_path.vertices[:, 1]
        print(f"Upper Bound Coordinates: {list(zip(upper_bound_x, upper_bound_y))}")
        
        # 计算下界
        for i in range(len(mid_line_x)):
            corresponding_upper_y = upper_bound_y[i] if i < len(upper_bound_y) else None
            if corresponding_upper_y is not None:
                lower_bound_y.append(2 * mid_line_y[i] - corresponding_upper_y)
            else:
                lower_bound_y.append(None)
                
        print(f"Calculated Lower Bound Coordinates (x same as Middle Line): {list(zip(mid_line_x, lower_bound_y))}")

        # 将结果保存到CSV文件
        results_df = pd.DataFrame({
            'Feature_Value': mid_line_x,
            'Middle_Line': mid_line_y,
            'Upper_Bound': upper_bound_y[:len(mid_line_x)],  # 确保长度一致
            'Calculated_Lower_Bound': lower_bound_y
        })
        results_df.to_csv(r'F:\第三篇论文写作\代码出图\PDP合并\FF2.csv', index=False)
        print("Results saved to confidence_intervals.csv")
    else:
        print("No Upper or Lower Bound Lines detected.")

# 假设model和X已经定义
selected_predictor = 'FF'
print('You selected', selected_predictor)
plot_pdp_and_calculate_bounds(model, X, selected_predictor)

In [ ]:
from pdpbox import pdp
import matplotlib.pyplot as plt
import pandas as pd

def plot_pdp_and_calculate_bounds(model, df, feature, cluster_flag=False, nb_clusters=None, lines_flag=False):
    # Create the data that we will plot
    pdp_goals = pdp.pdp_isolate(model=model, dataset=df, model_features=df.columns.tolist(), feature=feature)

    # Plot it without 'return_fig' parameter
    fig, ax = pdp.pdp_plot(pdp_goals, feature, cluster=cluster_flag, n_cluster_centers=nb_clusters, plot_lines=lines_flag, figsize=(10, 8))
    
    # 获取当前的matplotlib figure对象
    fig = plt.gcf()
    ax = plt.gca()
    
    # 修改标签字体大小和字体为新罗马
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(18)  # 设置字体大小
        item.set_fontname('Times New Roman')  # 设置字体为新罗马
    
    # 如果存在图例，则也需要修改其字体
    if ax.get_legend() is not None:
        plt.setp(ax.get_legend().get_texts(), fontsize='14')  # 设置图例字体大小
        plt.setp(ax.get_legend().get_texts(), fontname='Times New Roman')  # 设置图例字体为新罗马

    # 保存高质量的图像文件
    plt.savefig('output.png', dpi=1000)
    
    # 显示图像
    plt.show()

    # 提取坐标点
    print("Extracting coordinates of the lines...")
    
    # 获取中间线的坐标点
    mid_line_x = ax.lines[0].get_xdata()
    mid_line_y = ax.lines[0].get_ydata()
    print(f"Middle Line Coordinates: {list(zip(mid_line_x, mid_line_y))}")
    
    upper_bound_x, upper_bound_y = [], []
    lower_bound_y = []  # 初始化下界y坐标列表
    
    if len(ax.collections) > 0:
        upper_bound_path = ax.collections[0].get_paths()[0]
        upper_bound_x = upper_bound_path.vertices[:, 0]
        upper_bound_y = upper_bound_path.vertices[:, 1]
        print(f"Upper Bound Coordinates: {list(zip(upper_bound_x, upper_bound_y))}")
        
        # 计算下界
        for i in range(len(mid_line_x)):
            corresponding_upper_y = upper_bound_y[i] if i < len(upper_bound_y) else None
            if corresponding_upper_y is not None:
                lower_bound_y.append(2 * mid_line_y[i] - corresponding_upper_y)
            else:
                lower_bound_y.append(None)
                
        print(f"Calculated Lower Bound Coordinates (x same as Middle Line): {list(zip(mid_line_x, lower_bound_y))}")

        # 将结果保存到CSV文件
        results_df = pd.DataFrame({
            'Feature_Value': mid_line_x,
            'Middle_Line': mid_line_y,
            'Upper_Bound': upper_bound_y[:len(mid_line_x)],  # 确保长度一致
            'Calculated_Lower_Bound': lower_bound_y
        })
        results_df.to_csv(r'F:\第三篇论文写作\代码出图\PDP合并\AMMTmax2.csv', index=False)
        print("Results saved to confidence_intervals.csv")
    else:
        print("No Upper or Lower Bound Lines detected.")

# 假设model和X已经定义
selected_predictor = 'AMMTmax'
print('You selected', selected_predictor)
plot_pdp_and_calculate_bounds(model, X, selected_predictor)

In [ ]:
from pdpbox import pdp
import matplotlib.pyplot as plt
import pandas as pd

def plot_pdp_and_calculate_bounds(model, df, feature, cluster_flag=False, nb_clusters=None, lines_flag=False):
    # Create the data that we will plot
    pdp_goals = pdp.pdp_isolate(model=model, dataset=df, model_features=df.columns.tolist(), feature=feature)

    # Plot it without 'return_fig' parameter
    fig, ax = pdp.pdp_plot(pdp_goals, feature, cluster=cluster_flag, n_cluster_centers=nb_clusters, plot_lines=lines_flag, figsize=(10, 8))
    
    # 获取当前的matplotlib figure对象
    fig = plt.gcf()
    ax = plt.gca()
    
    # 修改标签字体大小和字体为新罗马
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(18)  # 设置字体大小
        item.set_fontname('Times New Roman')  # 设置字体为新罗马
    
    # 如果存在图例，则也需要修改其字体
    if ax.get_legend() is not None:
        plt.setp(ax.get_legend().get_texts(), fontsize='14')  # 设置图例字体大小
        plt.setp(ax.get_legend().get_texts(), fontname='Times New Roman')  # 设置图例字体为新罗马

    # 保存高质量的图像文件
    plt.savefig('output.png', dpi=1000)
    
    # 显示图像
    plt.show()

    # 提取坐标点
    print("Extracting coordinates of the lines...")
    
    # 获取中间线的坐标点
    mid_line_x = ax.lines[0].get_xdata()
    mid_line_y = ax.lines[0].get_ydata()
    print(f"Middle Line Coordinates: {list(zip(mid_line_x, mid_line_y))}")
    
    upper_bound_x, upper_bound_y = [], []
    lower_bound_y = []  # 初始化下界y坐标列表
    
    if len(ax.collections) > 0:
        upper_bound_path = ax.collections[0].get_paths()[0]
        upper_bound_x = upper_bound_path.vertices[:, 0]
        upper_bound_y = upper_bound_path.vertices[:, 1]
        print(f"Upper Bound Coordinates: {list(zip(upper_bound_x, upper_bound_y))}")
        
        # 计算下界
        for i in range(len(mid_line_x)):
            corresponding_upper_y = upper_bound_y[i] if i < len(upper_bound_y) else None
            if corresponding_upper_y is not None:
                lower_bound_y.append(2 * mid_line_y[i] - corresponding_upper_y)
            else:
                lower_bound_y.append(None)
                
        print(f"Calculated Lower Bound Coordinates (x same as Middle Line): {list(zip(mid_line_x, lower_bound_y))}")

        # 将结果保存到CSV文件
        results_df = pd.DataFrame({
            'Feature_Value': mid_line_x,
            'Middle_Line': mid_line_y,
            'Upper_Bound': upper_bound_y[:len(mid_line_x)],  # 确保长度一致
            'Calculated_Lower_Bound': lower_bound_y
        })
        results_df.to_csv(r'F:\第三篇论文写作\代码出图\PDP合并\AMT2.csv', index=False)
        print("Results saved to confidence_intervals.csv")
    else:
        print("No Upper or Lower Bound Lines detected.")

# 假设model和X已经定义
selected_predictor = 'AMT'
print('You selected', selected_predictor)
plot_pdp_and_calculate_bounds(model, X, selected_predictor)

In [ ]:
from pdpbox import pdp
import matplotlib.pyplot as plt
import pandas as pd

def plot_pdp_and_calculate_bounds(model, df, feature, cluster_flag=False, nb_clusters=None, lines_flag=False):
    # Create the data that we will plot
    pdp_goals = pdp.pdp_isolate(model=model, dataset=df, model_features=df.columns.tolist(), feature=feature)

    # Plot it without 'return_fig' parameter
    fig, ax = pdp.pdp_plot(pdp_goals, feature, cluster=cluster_flag, n_cluster_centers=nb_clusters, plot_lines=lines_flag, figsize=(10, 8))
    
    # 获取当前的matplotlib figure对象
    fig = plt.gcf()
    ax = plt.gca()
    
    # 修改标签字体大小和字体为新罗马
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(18)  # 设置字体大小
        item.set_fontname('Times New Roman')  # 设置字体为新罗马
    
    # 如果存在图例，则也需要修改其字体
    if ax.get_legend() is not None:
        plt.setp(ax.get_legend().get_texts(), fontsize='14')  # 设置图例字体大小
        plt.setp(ax.get_legend().get_texts(), fontname='Times New Roman')  # 设置图例字体为新罗马

    # 保存高质量的图像文件
    plt.savefig('output.png', dpi=1000)
    
    # 显示图像
    plt.show()

    # 提取坐标点
    print("Extracting coordinates of the lines...")
    
    # 获取中间线的坐标点
    mid_line_x = ax.lines[0].get_xdata()
    mid_line_y = ax.lines[0].get_ydata()
    print(f"Middle Line Coordinates: {list(zip(mid_line_x, mid_line_y))}")
    
    upper_bound_x, upper_bound_y = [], []
    lower_bound_y = []  # 初始化下界y坐标列表
    
    if len(ax.collections) > 0:
        upper_bound_path = ax.collections[0].get_paths()[0]
        upper_bound_x = upper_bound_path.vertices[:, 0]
        upper_bound_y = upper_bound_path.vertices[:, 1]
        print(f"Upper Bound Coordinates: {list(zip(upper_bound_x, upper_bound_y))}")
        
        # 计算下界
        for i in range(len(mid_line_x)):
            corresponding_upper_y = upper_bound_y[i] if i < len(upper_bound_y) else None
            if corresponding_upper_y is not None:
                lower_bound_y.append(2 * mid_line_y[i] - corresponding_upper_y)
            else:
                lower_bound_y.append(None)
                
        print(f"Calculated Lower Bound Coordinates (x same as Middle Line): {list(zip(mid_line_x, lower_bound_y))}")

        # 将结果保存到CSV文件
        results_df = pd.DataFrame({
            'Feature_Value': mid_line_x,
            'Middle_Line': mid_line_y,
            'Upper_Bound': upper_bound_y[:len(mid_line_x)],  # 确保长度一致
            'Calculated_Lower_Bound': lower_bound_y
        })
        results_df.to_csv(r'F:\第三篇论文写作\代码出图\PDP合并\RD2.csv', index=False)
        print("Results saved to confidence_intervals.csv")
    else:
        print("No Upper or Lower Bound Lines detected.")

# 假设model和X已经定义
selected_predictor = 'RD'
print('You selected', selected_predictor)
plot_pdp_and_calculate_bounds(model, X, selected_predictor)

In [ ]:
from pdpbox import pdp
import matplotlib.pyplot as plt
import pandas as pd

def plot_pdp_and_calculate_bounds(model, df, feature, cluster_flag=False, nb_clusters=None, lines_flag=False):
    # Create the data that we will plot
    pdp_goals = pdp.pdp_isolate(model=model, dataset=df, model_features=df.columns.tolist(), feature=feature)

    # Plot it without 'return_fig' parameter
    fig, ax = pdp.pdp_plot(pdp_goals, feature, cluster=cluster_flag, n_cluster_centers=nb_clusters, plot_lines=lines_flag, figsize=(10, 8))
    
    # 获取当前的matplotlib figure对象
    fig = plt.gcf()
    ax = plt.gca()
    
    # 修改标签字体大小和字体为新罗马
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(18)  # 设置字体大小
        item.set_fontname('Times New Roman')  # 设置字体为新罗马
    
    # 如果存在图例，则也需要修改其字体
    if ax.get_legend() is not None:
        plt.setp(ax.get_legend().get_texts(), fontsize='14')  # 设置图例字体大小
        plt.setp(ax.get_legend().get_texts(), fontname='Times New Roman')  # 设置图例字体为新罗马

    # 保存高质量的图像文件
    plt.savefig('output.png', dpi=1000)
    
    # 显示图像
    plt.show()

    # 提取坐标点
    print("Extracting coordinates of the lines...")
    
    # 获取中间线的坐标点
    mid_line_x = ax.lines[0].get_xdata()
    mid_line_y = ax.lines[0].get_ydata()
    print(f"Middle Line Coordinates: {list(zip(mid_line_x, mid_line_y))}")
    
    upper_bound_x, upper_bound_y = [], []
    lower_bound_y = []  # 初始化下界y坐标列表
    
    if len(ax.collections) > 0:
        upper_bound_path = ax.collections[0].get_paths()[0]
        upper_bound_x = upper_bound_path.vertices[:, 0]
        upper_bound_y = upper_bound_path.vertices[:, 1]
        print(f"Upper Bound Coordinates: {list(zip(upper_bound_x, upper_bound_y))}")
        
        # 计算下界
        for i in range(len(mid_line_x)):
            corresponding_upper_y = upper_bound_y[i] if i < len(upper_bound_y) else None
            if corresponding_upper_y is not None:
                lower_bound_y.append(2 * mid_line_y[i] - corresponding_upper_y)
            else:
                lower_bound_y.append(None)
                
        print(f"Calculated Lower Bound Coordinates (x same as Middle Line): {list(zip(mid_line_x, lower_bound_y))}")

        # 将结果保存到CSV文件
        results_df = pd.DataFrame({
            'Feature_Value': mid_line_x,
            'Middle_Line': mid_line_y,
            'Upper_Bound': upper_bound_y[:len(mid_line_x)],  # 确保长度一致
            'Calculated_Lower_Bound': lower_bound_y
        })
        results_df.to_csv(r'F:\第三篇论文写作\代码出图\PDP合并\MULT2.csv', index=False)
        print("Results saved to confidence_intervals.csv")
    else:
        print("No Upper or Lower Bound Lines detected.")

# 假设model和X已经定义
selected_predictor = 'MULT'
print('You selected', selected_predictor)
plot_pdp_and_calculate_bounds(model, X, selected_predictor)

In [ ]:
from pdpbox import pdp
import matplotlib.pyplot as plt
import pandas as pd

def plot_pdp_and_calculate_bounds(model, df, feature, cluster_flag=False, nb_clusters=None, lines_flag=False):
    # Create the data that we will plot
    pdp_goals = pdp.pdp_isolate(model=model, dataset=df, model_features=df.columns.tolist(), feature=feature)

    # Plot it without 'return_fig' parameter
    fig, ax = pdp.pdp_plot(pdp_goals, feature, cluster=cluster_flag, n_cluster_centers=nb_clusters, plot_lines=lines_flag, figsize=(10, 8))
    
    # 获取当前的matplotlib figure对象
    fig = plt.gcf()
    ax = plt.gca()
    
    # 修改标签字体大小和字体为新罗马
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(18)  # 设置字体大小
        item.set_fontname('Times New Roman')  # 设置字体为新罗马
    
    # 如果存在图例，则也需要修改其字体
    if ax.get_legend() is not None:
        plt.setp(ax.get_legend().get_texts(), fontsize='14')  # 设置图例字体大小
        plt.setp(ax.get_legend().get_texts(), fontname='Times New Roman')  # 设置图例字体为新罗马

    # 保存高质量的图像文件
    plt.savefig('output.png', dpi=1000)
    
    # 显示图像
    plt.show()

    # 提取坐标点
    print("Extracting coordinates of the lines...")
    
    # 获取中间线的坐标点
    mid_line_x = ax.lines[0].get_xdata()
    mid_line_y = ax.lines[0].get_ydata()
    print(f"Middle Line Coordinates: {list(zip(mid_line_x, mid_line_y))}")
    
    upper_bound_x, upper_bound_y = [], []
    lower_bound_y = []  # 初始化下界y坐标列表
    
    if len(ax.collections) > 0:
        upper_bound_path = ax.collections[0].get_paths()[0]
        upper_bound_x = upper_bound_path.vertices[:, 0]
        upper_bound_y = upper_bound_path.vertices[:, 1]
        print(f"Upper Bound Coordinates: {list(zip(upper_bound_x, upper_bound_y))}")
        
        # 计算下界
        for i in range(len(mid_line_x)):
            corresponding_upper_y = upper_bound_y[i] if i < len(upper_bound_y) else None
            if corresponding_upper_y is not None:
                lower_bound_y.append(2 * mid_line_y[i] - corresponding_upper_y)
            else:
                lower_bound_y.append(None)
                
        print(f"Calculated Lower Bound Coordinates (x same as Middle Line): {list(zip(mid_line_x, lower_bound_y))}")

        # 将结果保存到CSV文件
        results_df = pd.DataFrame({
            'Feature_Value': mid_line_x,
            'Middle_Line': mid_line_y,
            'Upper_Bound': upper_bound_y[:len(mid_line_x)],  # 确保长度一致
            'Calculated_Lower_Bound': lower_bound_y
        })
        results_df.to_csv(r'F:\第三篇论文写作\代码出图\PDP合并\AAS2.csv', index=False)
        print("Results saved to confidence_intervals.csv")
    else:
        print("No Upper or Lower Bound Lines detected.")

# 假设model和X已经定义
selected_predictor = 'AAS'
print('You selected', selected_predictor)
plot_pdp_and_calculate_bounds(model, X, selected_predictor)

In [ ]:
from pdpbox import pdp
import matplotlib.pyplot as plt
import pandas as pd

def plot_pdp_and_calculate_bounds(model, df, feature, cluster_flag=False, nb_clusters=None, lines_flag=False):
    # Create the data that we will plot
    pdp_goals = pdp.pdp_isolate(model=model, dataset=df, model_features=df.columns.tolist(), feature=feature)

    # Plot it without 'return_fig' parameter
    fig, ax = pdp.pdp_plot(pdp_goals, feature, cluster=cluster_flag, n_cluster_centers=nb_clusters, plot_lines=lines_flag, figsize=(10, 8))
    
    # 获取当前的matplotlib figure对象
    fig = plt.gcf()
    ax = plt.gca()
    
    # 修改标签字体大小和字体为新罗马
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(18)  # 设置字体大小
        item.set_fontname('Times New Roman')  # 设置字体为新罗马
    
    # 如果存在图例，则也需要修改其字体
    if ax.get_legend() is not None:
        plt.setp(ax.get_legend().get_texts(), fontsize='14')  # 设置图例字体大小
        plt.setp(ax.get_legend().get_texts(), fontname='Times New Roman')  # 设置图例字体为新罗马

    # 保存高质量的图像文件
    plt.savefig('output.png', dpi=1000)
    
    # 显示图像
    plt.show()

    # 提取坐标点
    print("Extracting coordinates of the lines...")
    
    # 获取中间线的坐标点
    mid_line_x = ax.lines[0].get_xdata()
    mid_line_y = ax.lines[0].get_ydata()
    print(f"Middle Line Coordinates: {list(zip(mid_line_x, mid_line_y))}")
    
    upper_bound_x, upper_bound_y = [], []
    lower_bound_y = []  # 初始化下界y坐标列表
    
    if len(ax.collections) > 0:
        upper_bound_path = ax.collections[0].get_paths()[0]
        upper_bound_x = upper_bound_path.vertices[:, 0]
        upper_bound_y = upper_bound_path.vertices[:, 1]
        print(f"Upper Bound Coordinates: {list(zip(upper_bound_x, upper_bound_y))}")
        
        # 计算下界
        for i in range(len(mid_line_x)):
            corresponding_upper_y = upper_bound_y[i] if i < len(upper_bound_y) else None
            if corresponding_upper_y is not None:
                lower_bound_y.append(2 * mid_line_y[i] - corresponding_upper_y)
            else:
                lower_bound_y.append(None)
                
        print(f"Calculated Lower Bound Coordinates (x same as Middle Line): {list(zip(mid_line_x, lower_bound_y))}")

        # 将结果保存到CSV文件
        results_df = pd.DataFrame({
            'Feature_Value': mid_line_x,
            'Middle_Line': mid_line_y,
            'Upper_Bound': upper_bound_y[:len(mid_line_x)],  # 确保长度一致
            'Calculated_Lower_Bound': lower_bound_y
        })
        results_df.to_csv(r'F:\第三篇论文写作\代码出图\PDP合并\ASWR2.csv', index=False)
        print("Results saved to confidence_intervals.csv")
    else:
        print("No Upper or Lower Bound Lines detected.")

# 假设model和X已经定义
selected_predictor = 'ASWR'
print('You selected', selected_predictor)
plot_pdp_and_calculate_bounds(model, X, selected_predictor)

In [ ]:
from pdpbox import pdp
import matplotlib.pyplot as plt
import pandas as pd

def plot_pdp_and_calculate_bounds(model, df, feature, cluster_flag=False, nb_clusters=None, lines_flag=False):
    # Create the data that we will plot
    pdp_goals = pdp.pdp_isolate(model=model, dataset=df, model_features=df.columns.tolist(), feature=feature)

    # Plot it without 'return_fig' parameter
    fig, ax = pdp.pdp_plot(pdp_goals, feature, cluster=cluster_flag, n_cluster_centers=nb_clusters, plot_lines=lines_flag, figsize=(10, 8))
    
    # 获取当前的matplotlib figure对象
    fig = plt.gcf()
    ax = plt.gca()
    
    # 修改标签字体大小和字体为新罗马
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(18)  # 设置字体大小
        item.set_fontname('Times New Roman')  # 设置字体为新罗马
    
    # 如果存在图例，则也需要修改其字体
    if ax.get_legend() is not None:
        plt.setp(ax.get_legend().get_texts(), fontsize='14')  # 设置图例字体大小
        plt.setp(ax.get_legend().get_texts(), fontname='Times New Roman')  # 设置图例字体为新罗马

    # 保存高质量的图像文件
    plt.savefig('output.png', dpi=1000)
    
    # 显示图像
    plt.show()

    # 提取坐标点
    print("Extracting coordinates of the lines...")
    
    # 获取中间线的坐标点
    mid_line_x = ax.lines[0].get_xdata()
    mid_line_y = ax.lines[0].get_ydata()
    print(f"Middle Line Coordinates: {list(zip(mid_line_x, mid_line_y))}")
    
    upper_bound_x, upper_bound_y = [], []
    lower_bound_y = []  # 初始化下界y坐标列表
    
    if len(ax.collections) > 0:
        upper_bound_path = ax.collections[0].get_paths()[0]
        upper_bound_x = upper_bound_path.vertices[:, 0]
        upper_bound_y = upper_bound_path.vertices[:, 1]
        print(f"Upper Bound Coordinates: {list(zip(upper_bound_x, upper_bound_y))}")
        
        # 计算下界
        for i in range(len(mid_line_x)):
            corresponding_upper_y = upper_bound_y[i] if i < len(upper_bound_y) else None
            if corresponding_upper_y is not None:
                lower_bound_y.append(2 * mid_line_y[i] - corresponding_upper_y)
            else:
                lower_bound_y.append(None)
                
        print(f"Calculated Lower Bound Coordinates (x same as Middle Line): {list(zip(mid_line_x, lower_bound_y))}")

        # 将结果保存到CSV文件
        results_df = pd.DataFrame({
            'Feature_Value': mid_line_x,
            'Middle_Line': mid_line_y,
            'Upper_Bound': upper_bound_y[:len(mid_line_x)],  # 确保长度一致
            'Calculated_Lower_Bound': lower_bound_y
        })
        results_df.to_csv(r'F:\第三篇论文写作\代码出图\PDP合并\DGF2.csv', index=False)
        print("Results saved to confidence_intervals.csv")
    else:
        print("No Upper or Lower Bound Lines detected.")

# 假设model和X已经定义
selected_predictor = 'DGF'
print('You selected', selected_predictor)
plot_pdp_and_calculate_bounds(model, X, selected_predictor)

In [ ]:
from pdpbox import pdp
import matplotlib.pyplot as plt
import pandas as pd

def plot_pdp_and_calculate_bounds(model, df, feature, cluster_flag=False, nb_clusters=None, lines_flag=False):
    # Create the data that we will plot
    pdp_goals = pdp.pdp_isolate(model=model, dataset=df, model_features=df.columns.tolist(), feature=feature)

    # Plot it without 'return_fig' parameter
    fig, ax = pdp.pdp_plot(pdp_goals, feature, cluster=cluster_flag, n_cluster_centers=nb_clusters, plot_lines=lines_flag, figsize=(10, 8))
    
    # 获取当前的matplotlib figure对象
    fig = plt.gcf()
    ax = plt.gca()
    
    # 修改标签字体大小和字体为新罗马
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(18)  # 设置字体大小
        item.set_fontname('Times New Roman')  # 设置字体为新罗马
    
    # 如果存在图例，则也需要修改其字体
    if ax.get_legend() is not None:
        plt.setp(ax.get_legend().get_texts(), fontsize='14')  # 设置图例字体大小
        plt.setp(ax.get_legend().get_texts(), fontname='Times New Roman')  # 设置图例字体为新罗马

    # 保存高质量的图像文件
    plt.savefig('output.png', dpi=1000)
    
    # 显示图像
    plt.show()

    # 提取坐标点
    print("Extracting coordinates of the lines...")
    
    # 获取中间线的坐标点
    mid_line_x = ax.lines[0].get_xdata()
    mid_line_y = ax.lines[0].get_ydata()
    print(f"Middle Line Coordinates: {list(zip(mid_line_x, mid_line_y))}")
    
    upper_bound_x, upper_bound_y = [], []
    lower_bound_y = []  # 初始化下界y坐标列表
    
    if len(ax.collections) > 0:
        upper_bound_path = ax.collections[0].get_paths()[0]
        upper_bound_x = upper_bound_path.vertices[:, 0]
        upper_bound_y = upper_bound_path.vertices[:, 1]
        print(f"Upper Bound Coordinates: {list(zip(upper_bound_x, upper_bound_y))}")
        
        # 计算下界
        for i in range(len(mid_line_x)):
            corresponding_upper_y = upper_bound_y[i] if i < len(upper_bound_y) else None
            if corresponding_upper_y is not None:
                lower_bound_y.append(2 * mid_line_y[i] - corresponding_upper_y)
            else:
                lower_bound_y.append(None)
                
        print(f"Calculated Lower Bound Coordinates (x same as Middle Line): {list(zip(mid_line_x, lower_bound_y))}")

        # 将结果保存到CSV文件
        results_df = pd.DataFrame({
            'Feature_Value': mid_line_x,
            'Middle_Line': mid_line_y,
            'Upper_Bound': upper_bound_y[:len(mid_line_x)],  # 确保长度一致
            'Calculated_Lower_Bound': lower_bound_y
        })
        results_df.to_csv(r'F:\第三篇论文写作\代码出图\PDP合并\PD2.csv', index=False)
        print("Results saved to confidence_intervals.csv")
    else:
        print("No Upper or Lower Bound Lines detected.")

# 假设model和X已经定义
selected_predictor = 'PD'
print('You selected', selected_predictor)
plot_pdp_and_calculate_bounds(model, X, selected_predictor)